# 1. <a id='toc1_'></a>[科学技術計算5](#toc0_)

ここでは固有値，特異値分解，主成分分析について説明する．

In [ ]:
import numpy as np
import scipy

from numpy.linalg import inv, norm, solve, det, matrix_rank, cond, pinv, lstsq, eig, eigh, eigvals, eigvalsh
from numpy import diag

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

from typing import List, Optional, Union, Tuple

import sklearn
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces

from matplotlib.pyplot import imshow


**目次**<a id='toc0_'></a>    
- 1. [科学技術計算5](#toc1_)    
  - 1.1. [固有値についての線形代数と数値計算のアプローチ](#toc1_1_)    
    - 1.1.1. [固有値と固有ベクトル](#toc1_1_1_)    
    - 1.1.2. [実対称行列の対角化](#toc1_1_2_)    
    - 1.1.3. [特性方程式](#toc1_1_3_)    
    - 1.1.4. [相似変換](#toc1_1_4_)    
    - 1.1.5. [数値的な解法](#toc1_1_5_)    
    - 1.1.6. [numpyとscipyの実装](#toc1_1_6_)    
    - 1.1.7. [固有値の例](#toc1_1_7_)    
      - 1.1.7.1. [ゲルシュゴリンの定理](#toc1_1_7_1_)    
    - 1.1.8. [固有ベクトルの例](#toc1_1_8_)    
  - 1.2. [べき乗法と関連手法](#toc1_2_)    
    - 1.2.1. [べき乗法（power method）](#toc1_2_1_)    
    - 1.2.2. [逆反復法（inverse iteration）](#toc1_2_2_)    
    - 1.2.3. [シフト付き逆反復法（shifted inverse iteration）](#toc1_2_3_)    
    - 1.2.4. [レイリー商反復（Rayleigh quotient iteration）](#toc1_2_4_)    
  - 1.3. [QR法](#toc1_3_)    
    - 1.3.1. [単純なQR分解の反復適用](#toc1_3_1_)    
    - 1.3.2. [ハウスホルダー法（Householder method）](#toc1_3_2_)    
      - 1.3.2.1. [鏡映変換](#toc1_3_2_1_)    
      - 1.3.2.2. [ハウスホルダー変換](#toc1_3_2_2_)    
        - 1.3.2.2.1. [1列目](#toc1_3_2_2_1_)    
        - 1.3.2.2.2. [2列目](#toc1_3_2_2_2_)    
        - 1.3.2.2.3. [$i=1,2,\ldots,n-2$列目](#toc1_3_2_2_3_)    
    - 1.3.3. [ギブンス変換](#toc1_3_3_)    
      - 1.3.3.1. [実装](#toc1_3_3_1_)    
      - 1.3.3.2. [ギブンス回転による三重対角行列の上三角化（QR分解）](#toc1_3_3_2_)    
    - 1.3.4. [QR法のまとめ](#toc1_3_4_)    
  - 1.4. [特異値分解（singular value decomposision）](#toc1_4_)    
    - 1.4.1. [固有値との関係](#toc1_4_1_)    
    - 1.4.2. [計算手法](#toc1_4_2_)    
    - 1.4.3. [numpyとscipyの実装](#toc1_4_3_)    
  - 1.5. [主成分分析（principal component analysis）](#toc1_5_)    
    - 1.5.1. [第1主成分](#toc1_5_1_)    
    - 1.5.2. [第$i$主成分](#toc1_5_2_)    
    - 1.5.3. [次元削減](#toc1_5_3_)    
    - 1.5.4. [累積寄与率](#toc1_5_4_)    
    - 1.5.5. [実装（$m > n$の場合）](#toc1_5_5_)    
    - 1.5.6. [scikit-learnによる実装](#toc1_5_6_)    
    - 1.5.7. [実装（$m < n$の場合）：顔画像データセットの主成分分析](#toc1_5_7_)    
      - 1.5.7.1. [固有顔](#toc1_5_7_1_)    
  - 1.6. [課題](#toc1_6_)    
    - 1.6.1. [課題05-1：QR法による固有値分解](#toc1_6_1_)    
    - 1.6.2. [課題05-2：ハウスホルダー変換によるQR分解](#toc1_6_2_)    
      - 1.6.2.1. [「一般の行列を三重対角行列に変換する」ためのハウスホルダー法の復習](#toc1_6_2_1_)    
      - 1.6.2.2. [「一般の行列をQR分解する」ためのハウスホルダー変換](#toc1_6_2_2_)    
      - 1.6.2.3. [課題](#toc1_6_2_3_)    
    - 1.6.3. [課題05-3：SVDによる主成分分析](#toc1_6_3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1.1. <a id='toc1_1_'></a>[固有値についての線形代数と数値計算のアプローチ](#toc0_)

### 1.1.1. <a id='toc1_1_1_'></a>[固有値と固有ベクトル](#toc0_)

ある$n$次正方行列$A \in R^{n \times n}$に対して
$$
A \boldsymbol{x} = \lambda \boldsymbol{x}
$$
を満たす実数$\lambda \in R$を行列$A$の固有値（eigenvalue），
$n$次元ベクトル$\boldsymbol{x} \in R^n$をその固有値に対応する固有ベクトル（eigenvector）と呼ぶ．
重複するものを含めると固有値は$n$個存在し，
$\lambda_1, \ldots, \lambda_n \in C$と書く．
実行列であっても一般に固有値は複素数であり，
対応する固有ベクトル$\boldsymbol{x}_1, \ldots, \boldsymbol{x}_n \in C^n$も複素ベクトルである．

実応用では実対称行列$A = A^T$を扱う場合が多く，その場合には固有値$\lambda_1, \ldots, \lambda_n \in R$はすべて実数となり，固有値ベクトル$\boldsymbol{x}_1, \ldots, \boldsymbol{x}_n \in R^n$も実数ベクトルとなる．
以下では主に実対称行列を扱う．

実対称行列の$n$個の固有値は実数であるので，大小関係を（重複するものは等号で）以下のように定義する．
$$
\lambda_1 \ge \lambda_2 \ge \cdots \ge \lambda_{n-1} \ge \lambda_n
$$
このうち$\lambda_1$を最大固有値，$\lambda_n$を最小固有値と呼び，
それぞれ$\lambda_\mathrm{max}, \lambda_\mathrm{min}$と書くこともある．
行列$A$の固有値という意味で，$\lambda_1(A), \ldots, \lambda_n(A)$
や$\lambda_\mathrm{max}(A), \lambda_\mathrm{min}(A)$と
書く場合もある．



### 1.1.2. <a id='toc1_1_2_'></a>[実対称行列の対角化](#toc0_)

実対称行列の異なる固有値$\lambda_i \neq \lambda_j$に対応する固有ベクトル$\boldsymbol{x}_i, \boldsymbol{x}_j$は直交するので，固有ベクトルを列に並べた行列
$$
U = [\boldsymbol{x}_1 \ \cdots \ \boldsymbol{x}_n] \in R^{n \times n}
$$
は直交行列$U U^T = U^T U = I$となる．

固有値を対角成分に並べた対角行列を
$$
D =
\begin{pmatrix}
\lambda_1 \\
& \lambda_2 \\
&& \ddots \\
&&& \lambda_n
\end{pmatrix}
 \in R^{n \times n}
$$
とすると，
\begin{align*}
A \boldsymbol{x}_1 &= \lambda_1 \boldsymbol{x}_1 \\
A \boldsymbol{x}_2 &= \lambda_2 \boldsymbol{x}_2 \\
\vdots \\
A \boldsymbol{x}_n &= \lambda_n \boldsymbol{x}_n \\
\end{align*}
なので，これらをまとめて
$$
AU = UD
$$
と表せる．すると行列$A$は次のように変形できる．
$$
U^T A U = D
$$
これを対角化（diagonalization）と呼ぶ．
また等価であるが行列$A$は次のように分解できる．
$$
A = U D U^T
$$
これを固有値分解（eigenvalue decomposition）と呼ぶ．
また数値解析においては実対称行列に対するシュール分解（symmetric Schur decomposition）とも呼ばれる
（一般の行列に対するシュール分解は$D$が対角行列ではなく上三角行列である）．

### 1.1.3. <a id='toc1_1_3_'></a>[特性方程式](#toc0_)

固有値分解（対角化）を行うには，固有値と固有ベクトルを求める．
まず
\begin{align*}
A \boldsymbol{x} &= \lambda \boldsymbol{x} \\
(A - \lambda I) \boldsymbol{x} &= \boldsymbol{0}
\end{align*}
だから，この斉次方程式が非自明な解を持つためには
$A - \lambda I$のランクが$n$未満，つまり
$$
\mathrm{det}(A - \lambda I) = 0
$$
である必要がある．これを特性方程式と呼ぶ．
これは$n$次多項式であり，その$n$個の根（つまり固有値）$\lambda_1, \ldots, \lambda_n$を求め，
それぞれについて
$\| \boldsymbol{x} \| = 1$という条件の元で
$A \boldsymbol{x} = \lambda_i \boldsymbol{x}$の解（固有ベクトル）を求める．

上記は線形代数で学ぶ手順であるが，数値解析の手段としては問題がある．$n>5$についての方程式には代数的解法が存在しないため，数値的に求める必要がある．また$n$次多項式は$n$が大きくなると，微小な変化に対して急激に値が変化するため，数値的に安定ではない．したがって，上記の手順以外の，数値的に安定な方法を用いる必要がある．

### 1.1.4. <a id='toc1_1_4_'></a>[相似変換](#toc0_)

実対称行列$A$に対して，直交行列$P$による次式の変換
$$
B = P^T A P
$$
を相似変換と呼び，$A$と$B$は相似であるといい，$A \sim B$と書くことがある．
次式のように，相似変換によって固有値は変化しない．
$$
B = P^T A P = P^T U^T D U P = U^{'T} D U'
$$
ここで$U' = UP$である．
したがって$A$の固有値を求める場合には，
相似変換によってできるだけ固有値計算が数値計算的に簡単で安定になるような行列$B$
へと変換するというアプローチを取ることが一般的である．

以下で説明するアプローチでは，
ハウスホルダー変換，ギブンス変換，QR変換が相似変換を求めるアルゴリズムであり，
計算しやすい行列$B$として上ヘッセンベルグ行列，上三角行列，三重対角行列などを利用する．

### 1.1.5. <a id='toc1_1_5_'></a>[数値的な解法](#toc0_)

対称行列の固有値と固有ベクトルを数値的に求める方法は多様な手法があり，また求めようとする行列$A$の性質によっても異なる．

- 1つの固有値とその固有ベクトルを求める反復方法
    - **べき乗法，逆反復法**
    - 固有値の近似値から開始する方法
        - **シフト付き逆反復法，レイリー商反復**
- すべての固有値と固有ベクトルを同時に反復的に求める方法
    - **QR法**
    - ヤコビ法
- 固有値を求めやすい行列（三重対角等）に変換する方法
    - **ハウスホルダー変換による三重対角化**
    - ギブンス変換による三重対角化
- 固有値を求めやすい行列（三重対角等）の固有値を求める方法
    - その後べき乗法などで固有ベクトルを計算
        - スツルム列（Strum sequence）を用いる二分法
            - 一部の固有値を選んで計算も可能
    - 同時に固有ベクトルも計算（対角化）
        - QR法
            - **ギブンス変換による三重対角行列の上三角化**
            - シフト（原点移動）を用いたQR法：現在標準的に使われている
        - 分割統治（divide-and-conquer）法：並列計算向き，現在標準的に使われている

これらは小規模な密の実対称行列に対する手法であるが，これら以外にも，
一般の非対称行列には上ヘッセンベルグ行列への変換とQR法，大規模疎行列にはランチョス法などがある．

一般的に言えることは以下のことである．

- 対称行列と非対称行列では計算手法が異なる．対称行列用のアルゴリズムのほうが計算効率が良い（計算量が少ない，必要メモリ量が少ない）．非対称行列用のアルゴリズムは対称行列にも適用できるが，不要な計算をしていることになる．
- 固有値を求めてから，固有ベクトルを求めるアルゴリズムが多い．固有値だけが必要なのに，固有ベクトルまで計算するアルゴリズムを用いるのは不要な計算である．

以下の説明では，上記の様々なアルゴリズムとアプローチのなかから，
まず1つの固有値と固有ベクトルを求める手法としてべき乗法とその関連手法を説明する．
次に，すべての固有値を求める手法としてQR変換を用いるQR法を説明する．
このQR変換は計算量の多いQR分解を用いるため，
QR分解の計算量の少ない行列へと変換するためのハウスホルダー変換とギブンス変換を説明する．

その前に，まずnumpyとscipyに実装されている固有値・固有ベクトル計算のソルバーを紹介する
（なお内部では分割統治法が用いられているが説明は省略する）．

### 1.1.6. <a id='toc1_1_6_'></a>[numpyとscipyの実装](#toc0_)

それぞれのライブラリで以下の関数が提供されている．
一般の行列用は，対称行列・非対称行列など任意の正方行列を扱う．
対称行列用の関数名の末尾の"h"はエルミート行列（Hermite，実行列の場合は対称行列）を表す．
これらはどれも（一部を除き）複素行列を扱える．

- 固有値のみを求める
    - 一般の行列の固有値を求める
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigvals.html
        - https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigvals.html
    - 対称行列の固有値を求める
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigvalsh.html
        - https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigvalsh.html
    - 実対称三重対角行列の固有値を求める
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigvalsh_tridiagonal.html
    - 対称帯行列の固有値を求める
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigvals_banded.html

- 固有値と固有ベクトルを求める
    - 一般の行列の固有値と固有ベクトルを求める
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html
        - https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html
    - 対称行列の固有値と固有ベクトルを求める
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html
        - https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html
    - 実対称三重対角行列の固有値と固有ベクトルを求める
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh_tridiagonal.html
    - 対称帯行列の固有値と固有ベクトルを求める
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig_banded.html



上記の関数の内部的には，LAPACKの関数を利用している．

- 一般行列用`eig()`
  - https://github.com/scipy/scipy/blob/f990b1d2471748c79bc4260baf8923db0a5248af/scipy/linalg/_decomp.py#L241
    - [dgeev](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dgeev.f)
- 対称行列用`eigh()`
  - https://github.com/scipy/scipy/blob/f990b1d2471748c79bc4260baf8923db0a5248af/scipy/linalg/_decomp.py#L546)
      - [dsyev](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dsyev.f), [dsyevd](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dsyevd.f), [dsyevr](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dsyevr.f), [dsyevx](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dsyevx.f)
- 実対称三重対角用`eigh_tridiagonal()`
  - https://github.com/scipy/scipy/blob/f990b1d2471748c79bc4260baf8923db0a5248af/scipy/linalg/_decomp.py#L1322C28-L1322C32
      - [dstev](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dstev.f)
- 対称帯行列用`eig_banded()`
  - https://github.com/scipy/scipy/blob/f990b1d2471748c79bc4260baf8923db0a5248af/scipy/linalg/_decomp.py#L794
    - [dsbevd](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dsbevd.f)

これらの中から，必要なものを選択して利用する必要がある．
たとえば実対称行列にもかかわらず一般の行列用の`eig()`を用いてしまうと，不要な計算をしているだけでなく，得られる固有値や固有ベクトルが複素数（実部に対して虚部の比率がマシンイプシロン程度の値）になってしまう場合もある．

以下で重要なのは対称行列用の`eigh()`と`eigvalsh()`である．
numpyとscipyのどちらを用いても良いが，numpyのほうがシンプルなインターフェースであり（つまり引数のオプションは少ない），scipyの方は一般固有値問題や変数上書きによるメモリ節約など高度なオプションが利用できる．

### 1.1.7. <a id='toc1_1_7_'></a>[固有値の例](#toc0_)

ここで対称行列$A$と一般の行列$B$について，固有値を求めてみる．

In [ ]:
m = 10
n = 5
A = np.random.rand(m, n)
A = A.T @ A
print("A\n", A, sep='')

n = 5
B = np.random.rand(n, n)
print("B\n", B, sep='')


以下がnumpyとscipyで固有値を求めるコードである．
固有値のみを求めるために，対称行列$A$には`eigvalsh()`を用い，非対称行列$B$には`eigvals()`を用いている．


In [ ]:
print("eigenavlues", np.linalg.eigvalsh(A))
print("eigenavlues", scipy.linalg.eigvalsh(A))


In [ ]:
with np.printoptions(
    formatter={
    'float': '{: <25.8f}'.format,
    'complexfloat': '{: <25.8f}'.format
    },
    linewidth=1000
):
    print("eigenavlues", np.linalg.eigvals(B))
    print("eigenavlues", scipy.linalg.eigvals(B))


なお対称行列$A$に`eigvals()`を用いると，実数であるべき固有値が複素数で求まる場合もある（ただし虚部は0かマシンイプシロン程度の小さい値）．対称行列では途中計算もすべて実数であるのにも関わらず，内部的には複素数で計算するため，無駄な計算をしてしまっている（可能性がある）ことに注意．

In [ ]:
with np.printoptions(
    formatter={
    'float': '{: <25.8f}'.format,
    'complexfloat': '{: <25.8f}'.format
    },
    linewidth=1000
):
    print("eigenavlues", np.linalg.eigvals(A))
    print("eigenavlues", scipy.linalg.eigvals(A))


#### 1.1.7.1. <a id='toc1_1_7_1_'></a>[ゲルシュゴリンの定理](#toc0_)

ゲルシュゴリンの定理によれば，（複素）正方行列$A \in C^{n \times n}$の（複素）固有値は，
中心が$a_{ii} \in C$で半径が$R_i = \sum_{j \neq i} |a_{ij}|$の円盤（Gershgorin' disk）$D_i \subset C$の
和集合$\cup_i D_i$に存在する．
実対称行列であれば実軸上だけで考えればよい．

- https://ja.wikipedia.org/wiki/%E3%82%B2%E3%83%AB%E3%82%B7%E3%83%A5%E3%82%B4%E3%83%AA%E3%83%B3%E3%81%AE%E5%AE%9A%E7%90%86

以下は，実対称行列$A$の実軸上のゲルシュゴリン円盤（つまり実軸上の区間）を求めている．

In [ ]:
print("A\n", A, sep='')

gershgorin_center = diag(A)  # 中心
gershgorin_radius = abs(A).sum(axis=1) - diag(A)  # 半径
gershgorin_set = np.stack((
    gershgorin_center + gershgorin_radius,
    gershgorin_center - gershgorin_radius
))
print("center", gershgorin_center)
print("radius", gershgorin_radius)
print("set\n", gershgorin_set, sep='')
print("tentative interval", gershgorin_set[0].max(), gershgorin_set[1].min())


これを可視化したものが以下のプロットである．赤い区間に（実）固有値が存在する．
赤の縦線が各区間の中心，
黒の縦線が実際の固有値である．
求まった固有値は，ゲルシュゴリンの定理で得られた区間内にあることが確認できる．
またこの定理は大まかな存在範囲を示しているだけであり，これでは数値計算でこの定理を利用できない．

In [ ]:
fig = plt.figure(figsize=(5, 1))
ax = fig.subplots()
ax.set_yticks([])

for i in range(gershgorin_set.shape[1]):
    ax.axvspan(
        gershgorin_set[0, i],
        gershgorin_set[1, i],
        alpha=0.1, color='red')

ax.vlines(
    gershgorin_center, 0, 1,
    colors="red")

d = np.linalg.eigvalsh(A)
ax.vlines(
    d, 0, 1,
    colors="black")

ax.set_xlabel("real axis")
plt.show()


以下は非対称行列$B$について，複素平面上でゲルシュゴリン円盤の中心と半径を求めている．

In [ ]:
print("B\n", B, sep='')

gershgorin_center = diag(B)  # 中心
gershgorin_radius = abs(B).sum(axis=1) - diag(B)  # 半径
print("center", gershgorin_center)
print("radius", gershgorin_radius)


これを可視化したものが以下のプロットである．赤い円盤中に（複素）固有値が存在する．
赤点が各区間の中心，
黒点が実際の固有値である．
求まった固有値は，ゲルシュゴリンの定理で得られた区間内にあることが確認できるが，
やはりこの定理は大まかな存在範囲を示しているだけである．

In [ ]:
import matplotlib.patches as patches

fig = plt.figure(figsize=(3, 3))
ax = fig.subplots()

for i in range(len(gershgorin_center)):
    c = patches.Circle(
        xy=(gershgorin_center[i], 0),
        radius=gershgorin_radius[i],
        fc='red', alpha=0.1)
    ax.add_patch(c)

ax.scatter(
    gershgorin_center,
    np.zeros_like(gershgorin_center),
    s=1, color="r")

d = np.linalg.eigvals(B)
d = np.array([(r.real, r.imag) for r in d])
ax.scatter(
    d[:, 0],
    d[:, 1],
    s=1, color="black")

ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)
ax.set_xlabel("Re")
ax.set_ylabel("Img")
ax.axis('equal')

plt.show()


### 1.1.8. <a id='toc1_1_8_'></a>[固有ベクトルの例](#toc0_)

次に固有ベクトルを求める．
固有値ベクトルを求めるために，対称行列$A$には`eigh()`を用い，非対称行列$B$には`eig()`を用いる．
どちらの関数でも，またnumpyでもscipyでも，
返り値は$n$個の固有値を並べたベクトル`d`のndarrayと，固有ベクトルからなる$n \times n$行列`U`のndarrayである．

なおこれらの関数は固有値と固有ベクトルを同時に求めるものであり，
「あらかじめ求めてある固有値を利用して固有ベクトルを求める」ものではないことに注意する必要がある．
実際には，固有値を求めるために`eigvals()`を用いて，次に固有ベクトルを求めるために`eig()`を用いると，
固有値を求める計算が重複するため無駄が生じている．

以下のコードでは，あらためて固有値と固有ベクトルを計算し，
固有値分解$A = U D U^T$もしくは対角化$D = U^T A U$が成立していることを確認している．
得られた固有値`d`はndarrayなので，`diag(d)`で対角行列$D$に変換して用いている
（実装上はブロードキャストを使ったほうが遥かに効率が良いが，ここでは数式に一致させている）．

In [ ]:
d, U = np.linalg.eigh(A)

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):

    print("eiganvalues", d)
    print("eiganvectors\n", U, sep='')

    print("U D U^T == A", np.allclose(U @ diag(d) @ U.T, A))
    print("U^T A U == D", np.allclose(U.T @ A @ U, diag(d)))


In [ ]:
d, U = np.linalg.eig(B)

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=200):

    print("eiganvalues", d)
    print("eiganvectors\n", U, sep='')

    print("U D U^{-1} == B", np.allclose(U @ diag(d) @ inv(U), B))
    print("D == U^{-1} B U", np.allclose(diag(d), inv(U) @ B @ U))


numpyとscipyの`eigh()`で得られる固有値と固有ベクトルの順番は昇順である（in ascending order）とドキュメントに書かれているので，
最大固有値と最小固有値，またそれぞれに対応する固有ベクトルは以下のように取得できる．

なお教科書などの一般的な固有値の表記では最大固有値が$\lambda_1$で最小固有値が$\lambda_n$であることが多いが，
ここでは逆になっており，最小固有値が`d[0]`で最大固有値が`d[-1]`（`d[n-1]`と等価）であることに注意．

In [ ]:
d, U = np.linalg.eigh(A)

lmd_max = d[-1]  # in ascending order
u_max = U[:, -1]
print("largest eigenvalue", lmd_max)
print("its eigenvextor", u_max)

lmd_min = d[0]  # in ascending order
u_min = U[:, 0]
print("smallest eigenvalue", lmd_min)
print("its eigenvextor", u_min)


なお実対称行列を扱うこれ以降のコードでは，最大固有値が`d[0]`に来るように，`d`の要素と`U`の列を逆順に入れ替えることも行う．

In [ ]:
d, U = np.linalg.eigh(A)
d = d[::-1]
U = U[:, ::-1]

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):

    print("eiganvalues", d)
    print("eiganvectors\n", U, sep='')

    print("U D U^T == A", np.allclose(U @ diag(d) @ U.T, A))
    print("U^T A U == D", np.allclose(U.T @ A @ U, diag(d)))


なおscipyの`eigh()`には，必要な固有値と固有ベクトルだけを計算できるように引数`subset_by_index`がある．
これに`[a, b]`と指定すれば，固有値`d[a], d[a+1], ..., d[b-1], d[b]`と対応する固有ベクトルだけを計算できる．

In [ ]:
d, U = scipy.linalg.eigh(A, subset_by_index=[2, 4])

with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):

    print("eiganvalues", d)
    print("eiganvectors\n", U, sep='')

    print("U D U^T == A", np.allclose(U @ diag(d) @ U.T, A))
    print("U^T A U == D", np.allclose(U.T @ A @ U, diag(d)))


numpyやscipyなどのライブラリでを用いれば，これらの固有値・固有ベクトルの計算は1行で可能である．
その内容や計算方法，計算量を知るために，以下ではまず固有ベクトルを求めるべき乗法とその関連手法を説明する．

## 1.2. <a id='toc1_2_'></a>[べき乗法と関連手法](#toc0_)

### 1.2.1. <a id='toc1_2_1_'></a>[べき乗法（power method）](#toc0_)

べき乗法は最大固有値$\lambda_1$に対応する固有ベクトル$\boldsymbol{x}_1$を求める方法であり，
同時に最大固有値も求める．
固有ベクトルを1つしか求められない単純な方法であるが，
他の手法でまず固有値を求め，その後に固有ベクトルを求めるためにべき乗法を用いることが多い．
そのため頻繁に用いられる基本的な手法である．

べき乗法は反復法であり，任意の初期値$\boldsymbol{u}^{(0)} \in R^n$から開始して，次式で解を更新する方法である．
$$
\boldsymbol{u}^{(k)} = A \boldsymbol{u}^{(k - 1)}
$$
これは最大固有値に対応する固有ベクトルに収束する．
$$
\lim_{k \to \infty} \boldsymbol{u}^{(k)} = \boldsymbol{x}_1
$$

これは次のように導出できる．
$A$の固有値が
$$
|\lambda_1| > |\lambda_2| \ge \cdots \ge |\lambda_n|
$$
を満たすと仮定すると，$c_i = \boldsymbol{x}_i^T \boldsymbol{u}^{(0)}$として，
\begin{align*}
\boldsymbol{u}^{(k)}
&= A \boldsymbol{u}^{(k - 1)} \\
&= A^k \boldsymbol{u}^{(0)} \\
&= A^k \sum_{i=1}^n c_i \boldsymbol{x}_i \\
&= \sum_{i=1}^n c_i \lambda_i^k \boldsymbol{x}_i \\
&= \lambda_1^k \sum_{i=1}^n c_i \left(\frac{\lambda_i}{\lambda_1}\right)^k \boldsymbol{x}_i
\end{align*}
ここで$i=2,\ldots,n$について
$\left| \frac{\lambda_i}{\lambda_1} \right| < 1$
であるので，$k \to \infty$のとき
\begin{align*}
\boldsymbol{u}^{(k)} &\simeq \lambda_1^k c_1 \boldsymbol{x}_1
\end{align*}
となる．

実際の計算では$\lambda_1^k$によってオーバフローやアンダーフローが発生する可能性があるため，
各反復で正規化する．

- 初期値$\boldsymbol{u}^{(0)}$, $k=0$
- 収束するまで以下を反復
    - $\boldsymbol{u}^{'(k)} = A \boldsymbol{u}^{(k - 1)}$
    - $\boldsymbol{u}^{(k)} = \boldsymbol{u}^{'(k)} / \| \boldsymbol{u}^{'(k)} \|$
- $\boldsymbol{x}_1 = \boldsymbol{u}^{(k)}$

得られた固有ベクトル$\boldsymbol{x}_1$に対応する固有値$\lambda_1$は
\begin{align*}
A \boldsymbol{x}_1 &= \lambda_1 \boldsymbol{x}_1\\
\boldsymbol{x}_1^T A \boldsymbol{x}_1 &= \lambda_1 \| \boldsymbol{x}_1 \|_2^2 \\
\lambda_1 &= \frac{\boldsymbol{x}_1^T A \boldsymbol{x}_1}{\| \boldsymbol{x}_1 \|_2^2}
\end{align*}
で求められる．固有ベクトルを正規化しておけば分母は不要である．

初期値は任意で良いが，$\boldsymbol{u}^{(0)} = (1, 1, \ldots, 1)^T$などが使われることもある．


以下のコードはべき乗法の簡単な実装である．

In [ ]:
u = np.ones(n)

maxiter = 10
for i in range(maxiter):
    u = A @ u
    u /= norm(u)
    l = u.T @ A @ u
    with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
        print(f"{i:2d} {l:12.12f}", u)


これを関数にしたものが以下の実装である．
$A \boldsymbol{u}$の計算を再利用しており，
また計算量の大きい固有値計算はループ中には行わず最後に行っている．
終了判定は反復回数か，固有ベクトルの変更がなくなるまでとした．
なお固有ベクトルは向きに依存しないため，
$\boldsymbol{u}^{(k)}$と$\boldsymbol{u}^{(k-1)}$の符号が反転する場合がある．
そのため`u`と，`u_pre`または`-u_pre`とが近くなれば終了としている．

In [ ]:
def power_method(
    A: np.ndarray,
    maxiter: int = 5000,
) -> Tuple[float, np.ndarray]:
    """Power method for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        maxiter (int, optional): max iterations. Defaults to 5000.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    u = np.ones(n)

    Au = u
    u_pre = u.copy()
    for i in range(maxiter):
        u = Au
        u /= norm(u)
        Au = A @ u

        with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
            print(f"{i:2}", u)

        if np.allclose(u, u_pre) or np.allclose(u, -u_pre):
            break
        u_pre = u.copy()

    l = u.T @ Au
    return l, u


べき乗法の計算結果は以下の通りである．

In [ ]:
lmd_max, u_max = power_method(A)
lmd_max, u_max


これは`np.linalg.eigh()`の結果とかなり近い．
ただしこの収束判定では固有ベクトルの計算誤差は1e-8程度であるため，`np.allclose()`では場合によってはFalseになる場合もある．


In [ ]:
d, U = np.linalg.eigh(A)

lmd_max_np = d[-1]  # in ascending order
u_max_np = U[:, -1]

lmd_max_np, u_max_np


In [ ]:
with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
    print(abs(lmd_max - lmd_max_np), np.isclose(lmd_max, lmd_max_np))
    print(abs(u_max - u_max_np), np.allclose(u_max, u_max_np))
    print(abs(u_max + u_max_np), np.allclose(u_max, -u_max_np))


### 1.2.2. <a id='toc1_2_2_'></a>[逆反復法（inverse iteration）](#toc0_)

逆反復法は最小固有値$\lambda_n$に対応する固有ベクトル$\boldsymbol{x}_n$を求める方法であり，
同時に最小固有値も求める．
これは行列$A^{-1}$の固有値が$A$の固有値の逆数であることを利用している．
つまり
\begin{align*}
A \boldsymbol{x}_n &= \lambda_n \boldsymbol{x}_n \\
\boldsymbol{x}_n &= \lambda_n A^{-1} \boldsymbol{x}_n \\
\frac{1}{\lambda_n} \boldsymbol{x}_n &= A^{-1} \boldsymbol{x}_n
\end{align*}
であり，同じ固有ベクトル$\boldsymbol{x}_n$を持つ．


方法は，単に$A^{-1}$に対してべき乗法を適用するだけである．

$$
\boldsymbol{u}^{(k)} = A^{-1} \boldsymbol{u}^{(k - 1)}
$$


得られた固有ベクトルに対応する固有値は
\begin{align*}
A^{-1} \boldsymbol{x}_n &= \frac{1}{\lambda_n} \boldsymbol{x}_n \\
\boldsymbol{x}_n^T A^{-1} \boldsymbol{x}_n &= \frac{1}{\lambda_n} \| \boldsymbol{x}_n \|_2^2 \\
\frac{1}{\lambda_n} &= \frac{\boldsymbol{x}_n^T A^{-1} \boldsymbol{x}_n}{\| \boldsymbol{x}_n \|_2^2} \\
\lambda_n &= \frac{\| \boldsymbol{x}_n \|_2^2}{\boldsymbol{x}_n^T A^{-1} \boldsymbol{x}_n}
\end{align*}
で求められる．もしくは
\begin{align*}
\lambda_n &= \frac{\boldsymbol{x}_n^T A \boldsymbol{x}_n}{\| \boldsymbol{x}_n \|_2^2}
\end{align*}
のほうが$A^{-1}$が不要になる．


実際には逆行列を用いるのではなく，以下の連立方程式を解く．

$$
A \boldsymbol{u}^{(k)} = \boldsymbol{u}^{(k - 1)}
$$

この係数行列$A$が固定である連立方程式を反復して解くことになるため，
あらかじめ$A = P L U$とLU分解しておき，これを用いて
\begin{align*}
P L \boldsymbol{y} &= \boldsymbol{u}^{(k - 1)} \\
U \boldsymbol{u}^{(k)} &= \boldsymbol{y}
\end{align*}
を反復する．

以下のコードでは，LU分解には`scipy.linalg.lu_factor`と`scipy.linalg.lu_solve`を用いている．
行列`A`に対してはLU分解を最初に1回だけ行い，
毎回のループでは計算量の少ない連立方程式を解いている．
求めている固有ベクトル`u`は，元の行列$A$の最小固有値`l1`に対応するが，
$A^{-1}$の最大固有値`l2`にも対応する．
`l2`は`l1`の逆数である．

In [ ]:
LU, pivot = scipy.linalg.lu_factor(A)

u = np.ones(n)

maxiter = 15
for i in range(maxiter):
    u = scipy.linalg.lu_solve((LU, pivot), u)
    u /= norm(u)
    l1 = u.T @ A @ u
    l2 = u.T @ scipy.linalg.lu_solve((LU, pivot), u)

    with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
        print(f"{i:2d} {l1:12.12f} {1 / l2:12.12f} ", u)


これを関数にしたものが以下の実装である．

In [ ]:
def inverse_iteration(
    A: np.ndarray,
    maxiter: int = 5000,
) -> Tuple[float, np.ndarray]:
    """inverse iteration method for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        maxiter (int, optional): max iterations. Defaults to 5000.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    u = np.ones(n)

    LU, pivot = scipy.linalg.lu_factor(A)

    Au = u
    u_pre = u.copy()
    for i in range(maxiter):
        u = Au
        u /= norm(u)
        Au = scipy.linalg.lu_solve((LU, pivot), u)


        with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
            print(f"{i:2d}", u)

        if np.allclose(u, u_pre) or np.allclose(u, -u_pre):
            break
        u_pre = u.copy()

    l = u.T @ A @ u
    return l, u


In [ ]:
lmd_max, u_max = inverse_iteration(A)
lmd_max, u_max


In [ ]:
d, U = np.linalg.eigh(A)

lmd_max_np = d[0]  # in ascending order
u_max_np = U[:, 0]

lmd_max_np, u_max_np


In [ ]:
with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
    print(abs(lmd_max - lmd_max_np), np.isclose(lmd_max, lmd_max_np))
    print(abs(u_max - u_max_np), np.allclose(u_max, u_max_np))
    print(abs(u_max + u_max_np), np.allclose(u_max, -u_max_np))


### 1.2.3. <a id='toc1_2_3_'></a>[シフト付き逆反復法（shifted inverse iteration）](#toc0_)

ある固有値が近似的に求まっているときに逆反復法を用いると，その固有値に収束する．
固有値$\lambda_i$の近似値を$\mu$として，
行列$A - \mu I$に対して逆反復法を適用する．
この行列の固有値は，$A$の固有値を$\mu$だけシフトしたものである．

$$
\boldsymbol{u}^{(k)} = (A - \mu I)^{-1} \boldsymbol{u}^{(k - 1)}
$$

べき乗法と同様に考えると，最小固有値$\lambda_i - \mu$の項が
他の固有値$\lambda_j - \mu$（$j \neq i$）よりも小さければ
$$
\left| \frac{\lambda_j - \mu}{\lambda_i - \mu} \right| < 1
$$
となる．
したがって最小固有値$\lambda_i - \mu$が求まり，
固有ベクトルもそれに対応するものに収束する．

以下のコードは，$\mu = 0.5$にした場合の例である．
LU分解を行う行列は$A - \mu I$であり，
それに対して`scipy.linalg.lu_factor`と`scipy.linalg.lu_solve`を用いている．
求めている固有ベクトル`u`は，元の行列$A$の固有値`l1`に対応するが，
$A - \mu I$の固有値`l2`にも対応する．
`l2`の逆数を`mu`だけシフトしたものが`l1`である．


In [ ]:
u = np.ones(n)
mu = 0.5
print("mu", mu)

LU, pivot = scipy.linalg.lu_factor(A -  mu * np.eye(n))

maxiter = 10
for i in range(maxiter):
    u = scipy.linalg.lu_solve((LU, pivot), u)
    u /= norm(u)
    l1 = u.T @ A @ u
    l2 = u.T @ scipy.linalg.lu_solve((LU, pivot), u)

    with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
        print(f"{i:2d} {l1:12.12f} {1 / l2 + mu:12.12f} ", u)


これを関数にしたものが以下の実装である．

In [ ]:
def shifted_inverse_iteration(
    A: np.ndarray,
    mu: float,
    maxiter: int = 5000,
) -> Tuple[float, np.ndarray]:
    """Shifted inverse iteration for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        mu (float): initial eigenvalue
        maxiter (int, optional): max iterations. Defaults to 5000.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    u = np.ones(n)

    LU, pivot = scipy.linalg.lu_factor(A -  mu * np.eye(n))

    Au = u
    u_pre = u.copy()
    for i in range(maxiter):
        u = Au
        u /= norm(u)
        Au = scipy.linalg.lu_solve((LU, pivot), u)

        with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
            print(f"{i:2d}", u)

        if np.allclose(u, u_pre) or np.allclose(u, -u_pre):
            break
        u_pre = u.copy()

    l = u.T @ A @ u
    return l, u


In [ ]:
shifted_inverse_iteration(A, 0.5)


$A$の固有値と比較すると，確かに初期値に最も近い固有値が得られていることがわかる．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
    d, U = np.linalg.eigh(A)
    print("d", d)
    print("U\n", U, sep='')


### 1.2.4. <a id='toc1_2_4_'></a>[レイリー商反復（Rayleigh quotient iteration）](#toc0_)

レイリー商
$$
\frac{\boldsymbol{x}^T A \boldsymbol{x}}{\boldsymbol{x}^T \boldsymbol{x}}
$$
は，固有値の良い近似を与える．
そこで，シフト付き逆反復法の$\mu$を，毎回の反復でレイリー商で置き換えたものがレイリー商反復である．

毎回の反復で係数行列が変化しLU分解が使えないため計算量は多くなるが，
3次収束するために反復数は少なくなる．


In [ ]:
u = np.ones(n)
mu = 0.5
print("mu", mu)

maxiter = 10
for i in range(maxiter):

    u = solve(A - mu * np.eye(n), u)
    u /= norm(u)
    mu = u.T @ A @ u

    with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
        print(f"{i:2d} {mu:12.12f}", u)


これを関数にしたものが以下の実装である．

In [ ]:
def rayleigh_quotient_iteration(
    A: np.ndarray,
    mu: float,
    u: np.ndarray,
    maxiter: int = 5000,
) -> Tuple[float, np.ndarray]:
    """Rayleigh quotioent iteration for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        mu (float): initial eigenvalue
        u (np.ndarray): n-d initial eigenvector
        maxiter (int, optional): max iterations. Defaults to 50.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    assert len(u) == n

    u_pre = u.copy()
    for i in range(maxiter):
        u = solve(A - mu * np.eye(n), u)
        u /= norm(u)
        mu = u.T @ A @ u

        with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
            print(f"{i:2d}", u)

        if np.allclose(u, u_pre) or np.allclose(u, -u_pre):
            break
        u_pre = u.copy()

    return mu, u


これはレイリー商を用いない場合よりも反復回数が少なくなっていることがわかる（場合によっては非常に少なくなる）．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.18f}'.format}, suppress=True, linewidth=200):
    mu, u = rayleigh_quotient_iteration(A, mu=0.5, u=np.ones(n))
    print(mu)


In [ ]:
with np.printoptions(formatter={'float': '{: 12.18f}'.format}, suppress=True, linewidth=200):
    d, U = np.linalg.eigh(A)
    print("d", d)
    print("U\n", U, sep='')


## 1.3. <a id='toc1_3_'></a>[QR法](#toc0_)

固有値が分かれば（もしくは近似値が得られれば），レイリー商反復などで固有ベクトルを求めることができる（固有ベクトルの近似が得られていればそれを初期値にする）．
以下では固有値と固有ベクトルを求める反復手法の1つであるQR法を説明する．
これは$A$のQR分解を用いたQR変換を反復適用する手法である．

行列$A \in R^{n \times n}$に対するQR分解を
$$
A = QR
$$
とする．ここで$Q \in R^{n \times n}$は直交行列（$Q^T Q = Q Q^T = I$），
$R \in R^{n \times n}$は上三角行列である．


\begin{align*}
\begin{pmatrix}
a_{11} & a_{12} & a_{13} & \cdots & a_{1n} \\
a_{21} & a_{22} & a_{23} & \cdots & a_{2n} \\
a_{31} & a_{32} & a_{33} & \cdots & a_{3n} \\
\vdots & \vdots &  & \ddots & \vdots \\
a_{n1} & a_{n2} & a_{n3} & \cdots & a_{nn}
\end{pmatrix}
=
\begin{pmatrix}
q_{11} & q_{12} & q_{13} & \cdots & q_{1n} \\
q_{21} & q_{22} & q_{23} & \cdots & q_{2n} \\
q_{31} & q_{32} & q_{33} & \cdots & q_{3n} \\
\vdots & \vdots &  & \ddots & \vdots \\
q_{n1} & q_{n2} & q_{n3} & \cdots & q_{nn}
\end{pmatrix}
\begin{pmatrix}
r_{11} & r_{12} & r_{13} & \cdots & r_{1n} \\
       & r_{22} & r_{23} & \cdots & r_{2n} \\
       &        & r_{33} & \cdots & r_{3n} \\
       &        &        & \ddots & \vdots \\
       &        &        &        & r_{nn}
\end{pmatrix}
\end{align*}

QR分解を行うアルゴリズムには以下のものがある．

- グラム・シュミット（Gram–Schmidt）の直交化法：任意の基底ベクトルを正規直交基底へ変換する方法である．すでに求めた直交基底が張る部分空間への射影との差分により直交基底を一つ増やすというアルゴリズムは理解しやすいため線形代数で学ぶが，数値計算的には作成された基底の直交性に問題があり，あまり用いられない．特に「射影との差分」が数値的に不安定である．（修正グラム・シュミット法もある）
- ハウスホルダー変換を用いる方法：ある列の対角成分より下の要素を鏡映変換で一度に0にする方法．以下では「一般の行列を三重対角行列に変換する」ハウスホルダー法を説明するが，それと同じ要領で「一般の行列をQR分解する」ためにハウスホルダー変換を行う．（ここでは説明は省略）
- ギブンス変換を用いる方法：ギブンス回転行列を適用することで，ある要素を一つ0にする方法．以下では「三重対角行列を上三角行列へ変換する」ギブンス変換を説明するが，それと同じ要領で「一般の行列をQR分解する」ためにギブンス変換を行う．（ここでは説明は省略）

ここではまず一般の行列に対するQR分解である`np.linalg.qr`（またはscipyの`scipy.linalg.qr`）を用いる．

- https://numpy.org/doc/stable/reference/generated/numpy.linalg.qr.html
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.qr.html

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    Q, R = np.linalg.qr(A)
    # Q, R = scipy.linalg.qr(A)
    print("Q\n", Q, sep='')
    print("R\n", R, sep='')
    print("QR == A", np.allclose(Q @ R, A))


実対称行列$A$に対してこのQR分解を繰り返し適用するのがQR法である．

- $A^{(0)} = A, k=0$
- 収束するまで以下を反復
    - QR分解を行う：$A^{(k)} = Q^{(k)} R^{(k)}$
    - $A^{(k+1)} \leftarrow R^{(k)} Q^{(k)}$

ここで
\begin{align*}
A^{(k)} &= Q^{(k)} R^{(k)} \\
A^{(k)} Q^{(k)} &= Q^{(k)} R^{(k)} Q^{(k)} \\
Q^{(k)T} A^{(k)} Q^{(k)} &=  R^{(k)} Q^{(k)}
\end{align*}
であるため，
\begin{align*}
    A^{(k+1)} &= R^{(k)} Q^{(k)} \\
    &= Q^{(k)T} A^{(k)} Q^{(k)} \\
    &= Q^{(k)T}Q^{(k-1)T} A^{(k-1)} Q^{(k-1)}Q^{(k)} \\
    & \vdots \\
    &= Q^{(k)T}Q^{(k-1)T} \cdots Q^{(0)T} A^{(0)} Q^{(0)} \cdots Q^{(k-1)}Q^{(k)} \\
    &= Q_X^T A^{(0)} Q_X
\end{align*}
ここで$Q_X = Q^{(0)} \cdots Q^{(k-1)}Q^{(k)}$は直交行列であり，
したがって$A$と$A^{(k+1)}$は相似であり（$A \sim A^{(k+1)}$），固有値は等しい．


$A$の固有値が
$$
|\lambda_1| > |\lambda_2| > \cdots > |\lambda_n| > 0
$$
を満たすと仮定すると，
この反復によって，$A^{(k)}$は
対角成分$a_{ii}$に固有値$\lambda_i$を持つ対角行列に収束する（証明は省略）．

\begin{align*}
\lim_{k \to \infty} A^{(k)}
=
\begin{pmatrix}
\lambda_1 & \\
       & \lambda_2 \\
       &        & \lambda_3 &  \\
       &        &        & \ddots &  \\
       &        &        &        & \lambda_n
\end{pmatrix}
\end{align*}


なお非対称行列の場合にもQR法は有効であるが，その場合には
対角行列ではなく上三角行列に収束する（そのため対角化ではなくシュール分解になる）．

一般の$n$次対称行列に対するQR分解の計算量は$O(n^3)$であり，毎回の反復でこれを計算するのは非常に計算量が大きい．
またQR法の収束は1次であるため，反復回数が多い．

QR法の各ステップの計算量を削減するために，以下の手順をとる．

1. まずハウスホルダー法で元の行列を三重対角行列に変換する（$O(n^3)$）．
    三重対角行列に対するQR変換では三重対角は保たれる．
2. 三重対角行列を上三角行列へ変換する（つまりQR分解を行う）にはギブンス変換を用いる（$O(n^2)$）．
3. さらに工夫をすることで（Wilkinson shiftなどで）QR法の反復の計算量を削減できる（Golub & Van Loan, Matirx Computation, 4th ed., 2013を参照）．


以下では，まず一般のQR分解を用いたQR法を説明する．
次に，上記ステップ1のハウスホルダー法で上三角行列に変換できることを説明する．
最後に，上記ステップ2のギブンス変換でQR分解ができることを説明する
（ステップ3は省略）．


### 1.3.1. <a id='toc1_3_1_'></a>[単純なQR分解の反復適用](#toc0_)

まず一般のQR分解である`np.linalg.qr`を用いて，QR法を確認する．
最初の2回の反復で$A_1, A_2$は以下のように求められる．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):

    A0 = A.copy()
    print("A0\n", A0, sep='')
    Qall = np.eye(A.shape[0])

    Q0, R0 = np.linalg.qr(A0)
    A1 = R0 @ Q0
    print("A1\n", A1, sep='')
    print("Q0.T A Q0\n", Q0.T @ A @ Q, sep='')
    Qall = Qall @ Q0
    print("Qx.T A Qx\n", Qall.T @ A @ Qall, sep='')

    Q1, R1 = np.linalg.qr(A1)
    A2 = R1 @ Q1
    print("A2\n", A2, sep='')
    print("Q1.T Q0.T A Q0 Q1\n", Q1.T @ Q0.T @ A @ Q0 @ Q1, sep='')
    Qall = Qall @ Q1
    print("Qx.T A Qx\n", Qall.T @ A @ Qall, sep='')


これを反復すると以下のようになる．収束が遅いため，10回程度の反復では下三角成分は収束しない（特に優対角成分と劣対角成分が残る）．

In [ ]:
A0 = A.copy()

Qall = np.eye(A0.shape[0])
maxiter = 10
for i in range(maxiter):
    Q, R = np.linalg.qr(A0)
    A0 = R @ Q
    Qall = Qall @ Q
    print()
    print(i, diag(A0))
    QtAQ = Qall.T @ A @ Qall

    print("Q^T A Q == A^(k+1)", np.allclose(QtAQ, A0))

    print(
        "max of abs of lower diagonals of Q^T A Q",
        np.abs(QtAQ[np.tril_indices(n, k=-1)]).max()
    )

    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        print("Q^T A Q\n", QtAQ, sep='')


これを関数にしたものが以下の実装である．

- `eigvalsh_by_qr_method()`は固有値だけを求める関数であり，
- `eigh_by_qr_method()`は固有値と固有ベクトルを求める関数である．

どちらもアルゴリズムは同じであるが，$Q_X$（`Qall`）を計算し保持する部分が異なる．

終了条件は対角成分である固有値が変化しなくなるまで，とすると下三角成分はまだ0に収束していない．
そこで優対角成分も変化しなくなるまでとしている．

なお引数`qr_decomp`にはQR分解ソルバーを指定する．
デフォルトでは`np.linalg.qr`であるが，
入出力が同じ自作ソルバーを使うこともできる．

In [ ]:
def eigvalsh_by_qr_method(
    A0: np.ndarray,
    maxiter: int = 1000,
    qr_decomp: callable = np.linalg.qr
) -> Tuple[np.ndarray, np.ndarray]:
    """Computing eigenvalues and eigenvectors by QR decompoition

    Args:
        A0 (np.ndarray): nxn matrix
        maxiter (int, optional): max iterations. Defaults to 1000.
        qr_decomp (callable, optional): QR decomposition solver. Defaults to np.linalg.qr.

    Returns:
        (np.ndarray) : n-d vector of eigenvalues
    """
    assert A0.ndim == 2
    assert A0.shape[0] == A0.shape[1]
    A = A0.copy()

    lmd_pre = diag(A)
    lmd1_pre = diag(A, k=1)

    for i in range(maxiter):
        Q, R = qr_decomp(A)
        A = R @ Q

        lmd = diag(A)
        lmd1 = diag(A, k=1)
        with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
            print(f"lmd  {i:2d}", lmd)
            print(f"lmd1 {i:2d}", lmd1)

        if np.allclose(lmd, lmd_pre) and np.allclose(lmd1, lmd1_pre):
            with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
                print("A\n", A, sep='')
            break

        lmd_pre = lmd.copy()
        lmd1_pre = lmd1.copy()

    return lmd


In [ ]:
def eigh_by_qr_method(
    A0: np.ndarray,
    maxiter: int = 1000,
    qr_decomp: callable = np.linalg.qr
) -> Tuple[np.ndarray, np.ndarray]:
    """Computing eigenvalues and eigenvectors by QR decompoition

    Args:
        A0 (np.ndarray): nxn matrix
        maxiter (int, optional): max iterations. Defaults to 1000.
        qr_decomp (callable, optional): QR decomposition solver. Defaults to np.linalg.qr.

    Returns:
        Tuple[np.ndarray, np.ndarray]: n-d vector of eigenvalues and orthogonal matrix with eigenvector columns
    """
    assert A0.ndim == 2
    assert A0.shape[0] == A0.shape[1]
    n = A0.shape[0]
    A = A0.copy()

    Qall = np.eye(n)
    lmd_pre = diag(A)
    lmd1_pre = diag(A, k=1)

    for i in range(maxiter):
        Q, R = qr_decomp(A)
        A = R @ Q
        Qall = Qall @ Q

        lmd = diag(A)
        lmd1 = diag(A, k=1)
        with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
            print(f"lmd  {i:2d}", lmd)
            print(f"lmd1 {i:2d}", lmd1)

        if np.allclose(lmd, lmd_pre) and np.allclose(lmd1, lmd1_pre):
            with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
                print("A\n", A, sep='')
            break

        lmd_pre = lmd.copy()
        lmd1_pre = lmd1.copy()

    return lmd, Qall


以下のコードは固有値だけを計算している．
またQR分解ソルバーを`scipy.linalg.qr`に変更しても結果は同じことが確認できる．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
    print("using numpy QR")
    d  = eigvalsh_by_qr_method(A)
    print(d)
    # print("using scipy QR")
    # d  = eigvalsh_by_qr_method(A, qr_decomp=scipy.linalg.qr)
    # print(d)


以下のように，この固有値は`np.linalg.eigh`の結果とほぼ一致する．
ただし`np.allclose()`では場合によってはFalseになる場合もある．

In [ ]:
d_np, _ = np.linalg.eigh(A)
d_np = d_np[::-1]

with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
    print("d_np", d_np)
    print("d   ", d)
    print("d == d_np", np.allclose(d, d_np))


以下のコードは固有値と固有ベクトルを計算し，対角化できているかを確認している．

なお

- $\mathrm{diag}(U^T A U) = (\lambda_1, \ldots, \lambda_n)^T$

つまりベクトルとして比較すると，固有値は精度良く求められていることが分かる．
また

- $U^T A U = \mathrm{diag}(\lambda_1, \ldots, \lambda_n)$

もしくは

- $A = U \mathrm{diag}(\lambda_1, \ldots, \lambda_n) U^T$

つまり行列として比較しても，`np.allclose()`による判定では一致していることがわかる（場合によっては計算誤差のためにFalseになる場合もある）．



In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    d, U = eigh_by_qr_method(A)
    print("d", d)
    print("U\n", U, sep='')

    # check
    print(
        "diag(U.T A U) == d", np.allclose(diag(U.T @ A @ U), d),
        "||diag(U.T A U) - d||", norm(diag(U.T @ A @ U) - d)
        )
    print(
        "U diag(d) U^T == A", np.allclose(U @ diag(d) @ U.T, A),
        "||U diag(d) U^T - A||", norm(U @ diag(d) @ U.T - A)
        )
    print("U diag(d) U^T - A\n", U @ diag(d) @ U.T - A, sep='')
    print(
        "U^T A U == diag(d)", np.allclose(U.T @ A @ U, diag(d)),
        "||U^T A U - diag(d)||", norm(U.T @ A @ U - diag(d))
        )
    print("U^T A U - diag(d)\n", U.T @ A @ U - diag(d), sep='')


以下のように`np.linalg.eigh`で得られる固有ベクトルと比較しても，ほぼ一致していることがわかる．

ただしQR法の反復はQR分解の計算量が多く反復回数も多いため，QR法では固有値だけを計算して，固有ベクトルはレイリー商反復などで求めるほうが，
計算量的にも（数本の固有ベクトルを求めるだけなら），計算精度的にもよい場合がある．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    d_np, U_np = np.linalg.eigh(A)
    U_np = U_np[:, ::-1]
    d_np = d_np[::-1]

    print("d_np", d_np)
    print("d   ", d)
    print("d == d_np", np.allclose(d, d_np))

    print("U_np\n", U_np, sep='')
    print("U\n", U, sep='')

    for i in range(len(U)):
        print(
            f"Are {i}th columns of U and U_np equal?:",
            np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )


### 1.3.2. <a id='toc1_3_2_'></a>[ハウスホルダー法（Householder method）](#toc0_)

直交行列はその行列式は$\pm 1$であるが，行列式が$-1$であるものを鏡映行列，
行列式が$+1$であるものを回転行列と呼ぶ．

ハウスホルダー法は，鏡映行列とベクトルの積をとると，
その結果のベクトルが1つの要素を除き0になるような鏡映行列（ハウスホルダー行列）を用いて，
$n-1$回の変換で$n$次対称行列を三重対角行列へ変換する方法である．

（なお次で説明するギブンス回転は，回転行列とベクトルの積を取ると，
その結果のベクトルのうち指定した1つの要素を0にするような回転行列を用いて変換を行う．）




#### 1.3.2.1. <a id='toc1_3_2_1_'></a>[鏡映変換](#toc0_)

単位ベクトル$\boldsymbol{u} \in R^n$を法線とする超平面に関する鏡映変換を表す行列$H \in R^{n \times n}$は次式で表される．

$$
H = I - 2 \boldsymbol{u} \boldsymbol{u}^T
$$

この$H$はハウスホルダー行列と呼ばれる．
あきらかに$H$は対称行列であり，したがって$H = H^T = H^{-1}$である．
ここで$\boldsymbol{x}, \boldsymbol{y} \in R^n$があって，
$\boldsymbol{x}$の鏡映変換をした結果が$\boldsymbol{y}$であるようにしたい，つまり

$$
\boldsymbol{y} = H \boldsymbol{x}
$$

としたい場合には，単位法線ベクトル$\boldsymbol{u}$として以下のようにすればよい．

$$
\boldsymbol{u} = \frac{\boldsymbol{x} - \boldsymbol{y}}{\| \boldsymbol{x} - \boldsymbol{y} \|}
$$


#### 1.3.2.2. <a id='toc1_3_2_2_'></a>[ハウスホルダー変換](#toc0_)

##### 1.3.2.2.1. <a id='toc1_3_2_2_1_'></a>[1列目](#toc0_)

まず実対称行列$A$の左から鏡映変換$Q_1$をかけて，$A$の第1列をできるだけ0にすることを考える．

\begin{align*}
Q_1
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
a_{31} & a_{32} & \cdots & a_{3n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{pmatrix}
=
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
* & * & \cdots & * \\
0 & * & \cdots & * \\
\vdots & \vdots & \ddots & \vdots \\
0 & * & \cdots & *
\end{pmatrix}
\end{align*}

ここで$*$は何らかの値（0ではない，が0でもよい任意の値）を意味する．
つまり$A$の第1行は保存し，第1列の第3要素以下を0にする．そのための行列が
\begin{align*}
Q_1 &=
\begin{pmatrix}
1 &  \\
 & H_1 \\
\end{pmatrix}
\in R^{n \times n}
\end{align*}
である．
これを$A$の左からかけると，$A$の第1行は保存され，
第2行以下だけがハウスホルダー行列$H_1 \in R^{n-1 \times n-1}$の変換を受ける．

ここで1列目の2行目以下を$n-1$次元ベクトル$\boldsymbol{x}$，
$A$の右下$(n-1) \times (n-1)$部分を$A_{n-1}$と書くと，
$A$は次のように表される．
\begin{align*}
A &=
\begin{pmatrix}
a_{11} & \boldsymbol{x}^T \\
\boldsymbol{x} & A_{n-1} \\
\end{pmatrix}
\end{align*}

ここでの目標は1列目2行目以下の$\boldsymbol{x}$の部分をハウスホルダー変換によってできるだけ0にすることであり，
$n-1$次元ベクトル$\boldsymbol{y}$として以下のように取ればよい．

\begin{align*}
\boldsymbol{x} &= (a_{21}, a_{31}, \ldots, a_{n1})^T \in R^{n-1} \\
\boldsymbol{y} &= (y_1, 0, \ldots, 0)^T \in R^{n-1}
\end{align*}

ここで$y_1$は定数である．
$\boldsymbol{y} = H \boldsymbol{x}$であり，直交変換はベクトルの長さを保存するため
（つまり$\| \boldsymbol{x} \| = \| \boldsymbol{y} \|$），
$y_1 = \pm \| \boldsymbol{x} \|$
とすればよい．
ただし
\begin{align*}
\boldsymbol{u} &= \frac{\boldsymbol{x} - \boldsymbol{y}}{\| \boldsymbol{x} - \boldsymbol{y} \|}
\propto
(a_{21} - y_1, a_{31}, \ldots, a_{n1})^T
\end{align*}
であり，第1要素で減算が生じるため，桁落ちを防ぐために
$a_{21}$とは逆の符号を用いる．つまり
$y_1 = - \mathrm{sgn}(a_{21}) \| \boldsymbol{x} \|$
とする．
これで以下が実現できる．
\begin{align*}
Q_1 A &=
\begin{pmatrix}
a_{11} & \boldsymbol{x}^T \\
\boldsymbol{y} & H_1 A_{n-1} \\
\end{pmatrix}
\end{align*}


$A$は対称行列であるので，$H$（つまり$Q_1$）を右からかけると，
第1行2列目以降の$\boldsymbol{x}^T$に対して同様に適用できる．
つまり

\begin{align*}
A &=
\begin{pmatrix}
a_{11} & \boldsymbol{x}^T \\
\boldsymbol{x} & A_{n-1} \\
\end{pmatrix}
\\
Q_1 A Q_1 &=
\begin{pmatrix}
a_{11} & (H_1 \boldsymbol{x})^T \\
H_1 \boldsymbol{x} & H_1 A_{n-1} H_1 \\
\end{pmatrix}
\\
 &=
\begin{pmatrix}
a_{11} & \boldsymbol{y}^T \\
\boldsymbol{y} & H_1 A_{n-1} H_1 \\
\end{pmatrix}
\\
&=
\left(
\begin{array}{c|c}
a_{11} &
\begin{array}{c}
y_1 & 0 & \cdots & 0
\end{array}
\\ \hline
\begin{array}{c}
y_1 \\ 0 \\ \vdots \\ 0
\end{array}
& H_1 A_{n-1} H_1
\end{array}
\right)
\end{align*}

これで第1列と第1行の要素は三重対角に変換されたことになる．



##### 1.3.2.2.2. <a id='toc1_3_2_2_2_'></a>[2列目](#toc0_)

次に$H_1 A_{n-1} H \in R^{n-1 \times n-1}$に対して同様の処理を行う．
そのためには
\begin{align*}
Q_2 &=
\begin{pmatrix}
1 &  \\
  & 1 \\
  &   & H_2 \\
\end{pmatrix}
\in R^{n \times n}
\end{align*}
を用いる．なお$H\in R^{n-2 \times n-2}$の部分の次元は$n-2$となり，反復毎に小さくなる．
これにより以下のようになる．
\begin{align*}
Q_2 Q_1 A Q_1 Q_2
&=
\left(
\begin{array}{c|c}
\begin{array}{cc}
a_{11} & y_1 \\
y_1 & a_{22}
\end{array}
&
\begin{array}{c}
 0 & 0 & \cdots & 0 \\
 y_2 & 0 & \cdots & 0 \\
\end{array}
\\ \hline
\begin{array}{cc}
0 & y_2 \\ 0 & 0 \\ \vdots & \vdots \\ 0 & 0
\end{array}
& H_2 [H_1 A_{n-1} H_1]_{n-2} H_2
\end{array}
\right)
\end{align*}
なおここで$[H_1 A_{n-1} H_1]_{n-2}$は$H_1 A_{n-1} H_1$の右下$(n-2) \times (n-2)$部分である．



##### 1.3.2.2.3. <a id='toc1_3_2_2_3_'></a>[$k=1,2,\ldots,n-2$列目](#toc0_)

これを$n-2$回繰り返すと，

$$
Q_{n-2} \cdots Q_2 Q_1 A Q_1 Q_2 \cdots Q_{n-2}
$$

は$A$を三重対角行列へ変換した行列になる．つまり

$$
Q_{\mathrm{hh}} = Q_1 Q_2 \cdots Q_{n-2}
$$

とすると

$$
A_{\mathrm{tri}} = Q_{\mathrm{hh}}^T A Q_{\mathrm{hh}}
$$

### 実装

以下はハウスホルダー変換の$k$回目の変換の実装である．

In [ ]:
def householder_step(
    A: np.ndarray,
    k: int
) -> Tuple[np.ndarray, np.ndarray]:
    """k-th step of Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix
        k (int): step

    Returns:
        Tuple[np.ndarray, np.ndarray]: transformed matrix QAQ after k-th step, and the transfrom Q
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]

    n = A.shape[0]
    assert 0 <= k < n - 2

    x = A[k + 1:, k]
    y = np.zeros_like(x)
    y[0] = - np.sign(A[k + 1, k]) * norm(x)

    print("x", x)
    print("y", y)

    u = (x - y) / norm(x - y)
    H = np.eye(len(x)) - 2 * np.outer(u, u)

    Q = np.eye(n)
    Q[k + 1:, k + 1:] = H

    QAQ = Q @ A @ Q
    return QAQ, Q


ハウスホルダー法は，これを各列についてn-2回繰り返す．


In [ ]:
def householder_transform(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
    """Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: tridiagonal Atri and orthogonal Qhh
                                       so that Qhh A Qhh == Atri
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    Atri = A.copy()
    Qhh = np.eye(n)
    for k in range(n - 2):
        print(f"{k}-th step")
        Atri, Q = householder_step(Atri, k)
        Qhh = Qhh @ Q
        print("A\n", Atri, sep='')
        print("Q\n", Q, sep='')
        print("Qhh\n", Qhh, sep='')
    return Atri, Qhh


以下のコードでは，行列`A`を三重対角行列`Atri`に変換しており，
そのための変換行列`Qhh`を得ている．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    Atri = A.copy()
    print("A\n", Atri, sep='')
    Atri, Qhh = householder_transform(Atri)
    print("tridiagonalized Atri\n", Atri, sep='')
    print("check Q.T A Q\n", Qhh.T @ A @ Qhh)
    print("Q.T A Q == Atri", np.allclose(Qhh.T @ A @ Qhh, Atri))


得られた三重対角行列`Atri`に対してQR分解を行いQR変換を行っても，三重対角が保たれていることが確認できる．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    Q, R = np.linalg.qr(Atri)
    print("Atri\n", Atri, sep='')
    print("Q\n", Q, sep='')
    print("R\n", R, sep='')
    print("RQ\n", R @ Q, sep='')


したがって，三重対角行列のQR分解の計算量が少なければ，QR法の計算量も少なくなる．
三重対角行列をQR分解するには，直交行列を左からかけて上三角行列にすればよい．
そのためには，三重対角行列

\begin{align*}
A &=
\begin{pmatrix}
a_{11} & a_{12} & 0 & 0 & \cdots & 0 & 0 & 0 \\
a_{21} & a_{22} & a_{23} & 0 & \cdots & 0 & 0 & 0 \\
0      & a_{32} & a_{33} & a_{34} & \cdots & 0 & 0 & 0 \\
0      & 0      & a_{43} & a_{44} & \cdots & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots  & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & a_{n-2, n-2} & a_{n-2, n-1} & 0\\
0 & 0 & 0 & 0 & \cdots & a_{n-1, n-2} & a_{n-1, n-1} & a_{n-1, n} \\
0 & 0 & 0 & 0 & \cdots & 0 & a_{n, n-1} & a_{nn} \\
\end{pmatrix}
\end{align*}

の$n-1$個の劣対角成分$a_{21}, a_{32}, \ldots, a_{n, n-1}$
を0にする直交変換を見つければよい．

これが以下に説明するギブンス変換であり，$n-1$個の対角成分を1つずつ0にするギブンス回転を$n-1$回適用する．


### 1.3.3. <a id='toc1_3_3_'></a>[ギブンス変換](#toc0_)

ハウスホルダー法は鏡映行列を利用して列ごとに要素の多くを一度に0に変換したが，
ギブンス変換はギブンス回転（Givens rotation）を利用して，
指定した1つの要素を0にするような変換を行う．

ギブンス回転行列$G$は次のようなものである．

\begin{align*}
G(p, q, \theta) =
\begin{pmatrix}
1 & \cdots & 0 & \cdots & 0 & \cdots & 0 \\
\vdots &  & \vdots &  & \vdots &  & \vdots \\
0 & \cdots & \cos\theta & \cdots & \sin\theta & \cdots & 0 \\
\vdots &  & \vdots &  & \vdots &  & \vdots \\
0 & \cdots & -\sin\theta & \cdots & \cos\theta & \cdots & 0 \\
\vdots &  & \vdots &  & \vdots &  & \vdots \\
0 & \cdots & 0 & \cdots & 0 & \cdots & 1 \\
\end{pmatrix}
\end{align*}

ここで回転行列$G$は，$p,q$行目と$p,q$列目（$p < q$）の4つの要素だけが$\sin$または$\cos$であり，その他の要素は単位行列と同じである．これは$p$-$q$平面上での2次元回転を表す（例えば3次元であれば$y$-$z$平面など）．
その2次元だけを考えて$2 \times 2$行列で書くと次のようになる．

\begin{align*}
R(\theta) =
\begin{pmatrix}
\cos\theta & \sin\theta  \\
-\sin\theta & \cos\theta \\
\end{pmatrix}
\end{align*}

この回転行列を適用して，片方の要素を0にしたい．つまり

\begin{align*}
R(\theta)^T
\begin{pmatrix}
x_p \\ x_q
\end{pmatrix}
=
\begin{pmatrix}
* \\ 0
\end{pmatrix}
\end{align*}

となる$\sin$と$\cos$を逆算する．これを用いて$G^T$を左から$A$にかければ，
$A$の$q$行$p$列目の要素を0にすることができる．
$n$次元ベクトル$\boldsymbol{x}, \boldsymbol{y}$に対する結果

\begin{align*}
\boldsymbol{y} = G(p, q, \theta) \boldsymbol{x}
\end{align*}

は次のようになる．

\begin{align*}
y_i &= x_i \quad (i \neq p, q) \\
y_p &= x_p \cos\theta - x_q \sin\theta \quad (i = p) \\
y_q &= x_p \sin\theta + x_q \cos\theta \quad (i = q) \\
\end{align*}

したがって，以下のようにすればギブンス回転行列を構成できる．

\begin{align*}
\cos\theta &= \frac{x_p}{\sqrt{x_p^2 + x_q^2}} \\
\sin\theta &= \frac{-x_q}{\sqrt{x_p^2 + x_q^2}} \\
\end{align*}




実際には$p$行目と$q$行目だけが影響を受けるため，その2行を抜き出して$2 \times 2$回転行列を適用すれば計算量を削減することができる．

ハウスホルダー法で対称行列を三重対角対称行列へ変換しているため，0にする目標要素は劣対角要素$a_{i+1, i}$である．これらをすべて0にして，かつ下三角成分を0に保てば，直交行列による上三角行列$R$への変換つまりQR分解が実現できる．

$a_{21}$を0にするギブンス回転$G(2, 1, \theta)$を$G_1$，
$a_{32}$を0にするギブンス回転$G(3, 2, \theta)$を$G_2$，
などと書くことにする
（つまり$a_{i+1, i}$を0にするギブンス回転$G(i+1, i, \theta)$を$G_i$と書く）と，
三重対角行列$A_{\mathrm{tri}}$に対して順次適用すれば以下のようになる．
\begin{align*}
G_{n-2}^T \cdots G_2^T G_1^T A_{\mathrm{tri}} &= R \\
Q^T A_{\mathrm{tri}} &= R \\
A_{\mathrm{tri}} &= Q R
\end{align*}
ここで$Q = G_1 G_2 \cdots G_{n-2}$である．

#### 1.3.3.1. <a id='toc1_3_3_1_'></a>[実装](#toc0_)

まず，ハウスホルダー法で得られた三重対角行列`Atri`を`AG`にコピーする．

In [ ]:
AG = Atri.copy()

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print(AG)


この行列の，2行1列の要素（`AG[1, 0]`）を0にしたい．そこで`q=1, p=0`とする．

In [ ]:
p, q = 0, 1


以下は$2 \times 2$ギブンス回転行列を作成する関数である．

In [ ]:
def givens_r_matrix(ab):
    """generate a matrix of Givens rotation
       from Gloub & Van Loan, Matrix Computation, 4th ed., 2013, Algorithm 5.1.3.

    Args:
        ab (ndarray): two values a, b

    Returns:
        ndarray: 2D rotation matrix
    """
    assert len(ab) == 2

    a, b = ab[0], ab[1]

    if abs(b) > abs(a):
        tau = -a / b
        s = 1 / np.sqrt(1 + tau**2)
        c = s * tau
    else:
        tau = -b / a
        c = 1 / np.sqrt(1 + tau**2)
        s = c * tau

    R = np.array([
        [c, s],
        [-s, c]
    ])
    return R


In [ ]:
def givens_r_matrix(
    xp_xq: np.ndarray
) -> np.ndarray:
    """generate a matrix of Givens rotation

    Args:
        xp_xq (ndarray): two values xp, xq

    Returns:
        ndarray: 2D rotation matrix
    """
    assert len(xp_xq) == 2

    xp, xq = xp_xq[0], xp_xq[1]

    # denom = np.sqrt(xp**2 + xq**2)
    denom = norm(xp_xq)
    c = xp / denom
    s = -xq / denom

    R = np.array([
        [c, s],
        [-s, c]
    ])
    return R


では$p$行目と$q$行目だけを抜き出して確認してみる．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print(AG[[p, q]])


実際にはこの2行のうちの$p$列目の2要素が必要となる．


In [ ]:
AG[[p, q]][:, p]


ではその2要素を用いてギブンス回転行列を構成する．

In [ ]:
Rot2x2 = givens_r_matrix(AG[[p, q]][:, p])
Rot2x2


この回転行列を$p$行目と$q$行目だけに適用した結果を確認すると，確かに$q$行$p$列の要素が0になっていることが分かる．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print(Rot2x2.T @ AG[[p, q]])


この結果を`AG`に上書きして全体の結果を見ると，
以下のよう$p$列目の劣対角成分が0にされて，上三角成分だけが残っていることが分かる．

In [ ]:
AG[[p, q]] = Rot2x2.T @ AG[[p, q]]

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print(AG)


以降は，この手順を$(p, q) = (1, 2), (2, 3), (3, 4), \ldots$について順次適用すればよい．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print("AG\n", AG, sep='')

    p, q = 1, 2
    print(f"AG[[{p}, {q}]]\n", AG[[p, q]], sep='')
    Rot2x2 = givens_r_matrix(AG[[p, q]][:, p])
    AG[[p, q]] = Rot2x2.T @ AG[[p, q]]
    print(f"R AG[[{p}, {q}]]\n", AG[[p, q]], sep='')
    print("AG\n", AG, sep='')

    p, q = 2, 3
    print(f"AG[[{p}, {q}]]\n", AG[[p, q]], sep='')
    Rot2x2 = givens_r_matrix(AG[[p, q]][:, p])
    AG[[p, q]] = Rot2x2.T @ AG[[p, q]]
    print(f"R AG[[{p}, {q}]]\n", AG[[p, q]], sep='')
    print("AG\n", AG, sep='')

    p, q = 3, 4
    print(f"AG[[{p}, {q}]]\n", AG[[p, q]], sep='')
    Rot2x2 = givens_r_matrix(AG[[p, q]][:, p])
    AG[[p, q]] = Rot2x2.T @ AG[[p, q]]
    print(f"R AG[[{p}, {q}]]\n", AG[[p, q]], sep='')
    print("AG\n", AG, sep='')


これで上三角行列$R$に変換はできたが，直交変換$G_1, G_2, \ldots$を得るためには，
$2 \times 2$回転行列を$n \times n$行列$Q$に格納する必要がある．

ここで$Q = G_1 G_2 \cdots G_{n-2}$であるので，逐次的に`Q = Q @ G`とすれば`Q`に格納できる．
しかし上で見たのと同様に，`G`を`Q`に右からかけると，`Q`の`p`列と`q`列にのみ影響するため，その2列を抜き出せばよい．

In [ ]:
AG = Atri.copy()
Q = np.eye(n)
k = 0


では直交行列$Q = G_1 G_2 \cdots$と上三角行列$R$を得るために，以下のセルを何度か実行して，手動でkについてのループを回し確認する．

なお`G`は実際には不要であるが，以下では確認のために`G`を表示している（advanced indexingを利用）．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print(f"k = {k} < {n-1} = n-1")
    assert k < n - 1
    p, q = k, k + 1
    print(f"p = {p}, q = {q}")

    Rot2x2 = givens_r_matrix(AG[[p, q]][:, p])

    print(f"AG[[{p}, {q}]]\n", AG[[p, q]], sep='')
    AG[[p, q]] = Rot2x2.T @ AG[[p, q]]
    print(f"R AG[[{p}, {q}]]\n", AG[[p, q]], sep='')
    print("AG\n", AG, sep='')

    G = np.eye(n)
    G[[[p, p], [q, q]],
      [[p, q], [p, q]]] = Rot2x2
    print("G\n", G, sep='')

    # Q = Q @ G
    Q[:, [p, q]] = Q[:, [p, q]] @ Rot2x2
    print("Q\n", Q, sep='')

    k = k + 1
    R = AG


確認すると，$A_{\mathrm{tri}} = QR$が求まっていることが分かる．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print("QR == Atri", np.allclose(Q @ R, Atri))


#### 1.3.3.2. <a id='toc1_3_3_2_'></a>[ギブンス回転による三重対角行列の上三角化（QR分解）](#toc0_)

以下が，三重対角行列に対するギブンス回転によるQR分解の関数である．

In [ ]:
def qr_decom_by_givens(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
    """QR decomposition of a tridiagonal matrix by Givens rotation

    Args:
        A (np.ndarray): an nxn tridiagonal matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: orthogonal Q and upper traiangular R so that A == QR
    """
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    # assert A should be tridiagonal
    n = A.shape[0]

    AG = A.copy()
    Q = np.eye(n)

    for k in range(n - 1):
        p, q = k, k + 1
        Rot2x2 = givens_r_matrix(AG[[p, q]][:, p])
        AG[[p, q]] = Rot2x2.T @ AG[[p, q]]
        Q[:, [p, q]] = Q[:, [p, q]] @ Rot2x2

    R = AG
    return Q, R


以下のコードはQR分解を確認している．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    Q_givens, R_givens = qr_decom_by_givens(Atri)
    print("Q\n", Q_givens, sep='')
    print("R\n", R_givens, sep='')
    print("Atri == QR", np.allclose(Atri, Q_givens @ R_givens))
    
    Q_np, R_np = np.linalg.qr(Atri)
    print("Q\n", Q_np, sep='')
    print("R\n", R_np, sep='')
    print("Q == Q_np", np.allclose(Q_givens, Q_np))
    print("R == R_np", np.allclose(R_givens, R_np))


### 1.3.4. <a id='toc1_3_4_'></a>[QR法のまとめ](#toc0_)

上記の手順をまとめると以下のようになる．

- ハウスホルダー変換により三重対角行列へ変換する
    - $A_{\mathrm{tri}} = Q_{\mathrm{hh}}^T A Q_{\mathrm{hh}}$
- $A^{(0)} = A_{\mathrm{tri}}, Q_X = I, k=0$
- 収束するまで以下を反復
    - ギブンス変換によるQR分解を行う：$A^{(k)} = Q^{(k)} R^{(k)}$
    - $A^{(k+1)} \leftarrow R^{(k)} Q^{(k)}$
    - $Q_X \leftarrow Q_X Q^{(k)}$

これで対角行列

- $D = A^{(k+1)}$

が得られ，$A_{\mathrm{tri}}$の対角化が得られた．

\begin{align*}
    D &= Q_X^T A_{\mathrm{tri}} Q_X \\
    A_{\mathrm{tri}} &= Q_X D Q_X^T
\end{align*}

これから$A$の対角化が得られる．

\begin{align*}
    A_{\mathrm{tri}} &= Q_{\mathrm{hh}}^T A Q_{\mathrm{hh}} \\
    A &= Q_{\mathrm{hh}} A_{\mathrm{tri}} Q_{\mathrm{hh}}^T \\
      &= Q_{\mathrm{hh}} Q_X D Q_X^T Q_{\mathrm{hh}}^T \\
\end{align*}

以下のコードは$A_{\mathrm{tri}}$の対角化を行っている．


In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    Atri, Qhh = householder_transform(A)
    print("Atri\n", Atri, sep='')
    print("Atri == Qhh.T A Qhh", np.allclose(Atri, Qhh.T @ A @ Qhh))

    d, Qx = eigh_by_qr_method(Atri, qr_decomp=qr_decom_by_givens)
    print("d", d)
    print("Qx\n", Qx, sep='')

    print("Qx D Qx^T\n", Qx @ diag(d) @ Qx.T, sep='')
    print("Qx^T Atri Qx\n", Qx.T @ Atri @ Qx, sep='')

    print("Qx^T Atri Qx == D", np.allclose(Qx.T @ Atri @ Qx, diag(d)))
    print("Qx D Qx^T == Atri", np.allclose(Qx @ diag(d) @ Qx.T, Atri))

    print("|| Qx D Qx^T - Atri||", np.linalg.norm(Qx @ diag(d) @ Qx.T - Atri))


これで得られた$A$の固有ベクトル$U = Q_{\mathrm{hh}} Q_X$は`np.linalg.eigh`で得られる固有ベクトルと比較してもほぼ一致していることがわかる．

ただし計算誤差のため，`np.allclose`で一致しない場合もある．したがって，これを初期値としてレイリー商反復などを行うほうが精度はよい．

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    d_np, U_np = np.linalg.eigh(A)
    U_np = U_np[:, ::-1]
    d_np = d_np[::-1]

    print("d_np", d_np)
    print("d   ", d)
    print("d == d_np", np.allclose(d, d_np))

    print("U_np\n", U_np, sep='')

    U = Qhh @ Qx
    print("U\n", U, sep='')

    for i in range(len(U)):
        print(
            f"Are {i}th columns of U and U_np equal?:",
            np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )


## 1.4. <a id='toc1_4_'></a>[特異値分解（singular value decomposision）](#toc0_)

実対称行列$A \in R^{n \times n}$に対する固有値分解は
$$
A = U D U^T
$$
であるが，非正方行列に対する分解が特異値分解である．

一般の実行列$A \in R^{m \times n}$について
$$
A = U \Sigma V^T
$$
を$A$の特異値分解と呼ぶ．
ここで$U \in R^{m \times m}$と
$V \in R^{n \times n}$は直交行列である．
それぞれ列ベクトルで表すと
\begin{align*}
U &= (\boldsymbol{u}_1, \ldots, \boldsymbol{u}_m) \\
V &= (\boldsymbol{v}_1, \ldots, \boldsymbol{v}_n)
\end{align*}
であり，$\boldsymbol{u}_i \in R^m$を左特異ベクトル，
$\boldsymbol{v}_i \in R^n$を右特異ベクトルと呼ぶ．


また$p=\mathrm{min}(m, n)$として
$$
\Sigma = \mathrm{diag}(\sigma_1, \ldots, \sigma_p) \in R^{m \times n}
$$
は対角行列と零行列からなる行列であり，対角部分の対角要素$\sigma_i$を特異値（singular values）と呼ぶ．

具体的には，
$m=n$なら
$$
\Sigma =
\begin{pmatrix}
\sigma_1 \\
& \sigma_2 \\
&& \ddots \\
&&& \sigma_n
\end{pmatrix}
$$

$m>n$なら
$$
\Sigma =
\left(
\begin{array}{c}
\begin{array}{cccc}
\sigma_1 \\
& \sigma_2 \\
&& \ddots \\
&&& \sigma_n \\
\end{array}
\\ \hline
\begin{array}{ccc}
\\
& 0 \\
\\
\end{array}
\end{array}
\right)
$$

$m<n$なら
$$
\Sigma =
\left(
\begin{array}{c|c}
\begin{array}{cccc}
\sigma_1 \\
& \sigma_2 \\
&& \ddots \\
&&& \sigma_m \\
\end{array}
&
\begin{array}{ccc}
\\
& 0 \\
\\
\end{array}
\end{array}
\right)
$$
である．

$m \neq n$の場合には$\Sigma$には零行列部分を含むため，$U$と$V$に不要な部分が生じる．

$m > n$ならば
\begin{align*}
U \Sigma &=
(\boldsymbol{u}_1, \ldots, \boldsymbol{u}_m)
\left(
\begin{array}{c}
\begin{array}{cccc}
\sigma_1 \\
& \sigma_2 \\
&& \ddots \\
&&& \sigma_n \\
\end{array}
\\ \hline
\begin{array}{ccc}
\\
& 0 \\
\\
\end{array}
\end{array}
\right) \\
&=
(\sigma_1 \boldsymbol{u}_1, \ldots, \sigma_n \boldsymbol{u}_n)
\end{align*}

同様にして$m < n$ならば

\begin{align*}
\Sigma V^T &=
\begin{pmatrix}
\sigma_m \boldsymbol{v}_1^T \\
\vdots \\
\sigma_m \boldsymbol{v}_m^T
\end{pmatrix}
\end{align*}

したがって，すべての特異ベクトル
$U \in R^{m \times m}$と$V \in R^{n \times n}$を求めずとも，
最初の$p=\mathrm{min}(m, n)$個の特異ベクトルだけを求めればよい．
この場合には
$U \in R^{m \times p}$と$V \in R^{p \times n}$となり，
また$\Sigma=\mathrm{diag}(\sigma_1, \ldots, \sigma_p) \in R^{p \times p}$である．

すべての特異ベクトルを求める方法をfull SVD，
必要な$p$個の特異ベクトルを求める方法をthin SVDなどと呼ぶ．
一般的な応用ではthin SVDを利用するだけで十分であり，特異値分解といえばthin SVDを指すことが多いが，
ライブラリの実装では計算量の多いfull SVDがデフォルトになっている場合があるので注意する必要がある．

### 1.4.1. <a id='toc1_4_1_'></a>[固有値との関係](#toc0_)

もし$A \in R^{m \times n}$によって対称行列が$A^T A \in R^{n \times n}$と表されるなら，

\begin{align*}
A^T A &= V \Sigma U^T U \Sigma V^T \\
&= V \Sigma^2 V^T
\end{align*}

となり，$A^T A$の固有値$\lambda_i$は$A$の特異値$\sigma_i$の2乗となることが分かる．
$m > n$なら

\begin{align*}
\begin{pmatrix}
\lambda_1 \\
& \lambda_2 \\
&& \ddots \\
&&& \lambda_n
\end{pmatrix}
=
\begin{pmatrix}
\sigma_1^2 \\
& \sigma_2^2 \\
&& \ddots \\
&&& \sigma_n^2
\end{pmatrix}
\end{align*}

であり，$m < n$なら

\begin{align*}
\begin{pmatrix}
\lambda_1 \\
& \lambda_2 \\
&& \ddots \\
&&& \lambda_n
\end{pmatrix}
=
\begin{pmatrix}
\sigma_1^2 \\
& \ddots \\
&& \sigma_m^2 \\
&&& 0 \\
&&&& \ddots \\
&&&&& 0 \\
\end{pmatrix}
\end{align*}

であるので$\lambda_{m+1}=\cdots=\lambda_n=0$である．


### 1.4.2. <a id='toc1_4_2_'></a>[計算手法](#toc0_)

様々な手法があるが，基本は固有値計算を利用している．
$m > n$の場合に行列$A \in R^{m \times n}$の特異値分解$A=U \Sigma V^T$を計算する単純な方法は以下のようなものである（実際にはもっと効率的で数値的に安定な方法が使われている）．

1. $A^T A$に対して対角化を計算し，$V^T (A^T A) V = \Sigma^2$を得る．
2. $A V$をQR分解して$AV = UR$を得る．ここで$U$は直交行列（QRのQ）である．$R$は上三角行列であるが，この場合には対角行列になるため（証明略），$A = U R V^T$が得られる．

### 1.4.3. <a id='toc1_4_3_'></a>[numpyとscipyの実装](#toc0_)

それぞれのライブラリで以下の関数が提供されている．

- 特異値のみを求める
    - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svdvals.html
    - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.diagsvd.html
- 特異値と特異ベクトルを求める
    - https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html
    - https://numpy.org/devdocs/reference/generated/numpy.linalg.svd.html

`svd()`のオプションには，特異ベクトルを求めるかどうかのフラグ（`compute_uv`）と，
full SVDかthin SVDかを選択するフラグ（`full_matrices`）がある．

返り値は，

- `full_matrices=True`（デフォルト）の場合には（full SVD），$m \times m$の$U$，$p$個の特異値からなるベクトル$s$，$n \times n$の$V^T$である（$V$ではないことに注意）．
- `full_matrices=False`の場合には（thin SVD），$m \times p$の$U$，$p$個の特異値からなるベクトル$s$，$p \times n$の$V^T$である．



In [ ]:
m = 10
n = 5
A = np.random.rand(m, n)

U, s, Vt = np.linalg.svd(A, full_matrices=False)
print("U Sigma V^T == A", np.allclose(U @ diag(s) @ Vt, A))

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print("A\n", A, sep='')
    print("U\n", U, sep='')
    print("V\n", Vt.T, sep='')
    print("s\n", s, sep='')


## 1.5. <a id='toc1_5_'></a>[主成分分析（principal component analysis）](#toc0_)

$n$次元のデータが$m$個ある場合に，
データ$\boldsymbol{x}_1, \boldsymbol{x}_2, \ldots, \boldsymbol{x}_m \in R^n$
を行に持つ行列

\begin{align*}
X =
\begin{pmatrix}
\boldsymbol{x}_1^T \\
\boldsymbol{x}_2^T \\
\vdots \\
\boldsymbol{x}_m^T
\end{pmatrix}
=
\begin{pmatrix}
x_{11} & x_{12} & \cdots & x_{1n} \\
x_{21} & x_{22} & \cdots & x_{2n} \\
\vdots \\
x_{m1} & x_{m2} & \cdots & x_{mn} \\
\end{pmatrix}
\in R^{m \times n}
\end{align*}

で表し，これをデータ行列やデザイン行列と呼ぶ事がある．

以下は有名なアヤメのデータセットを`X`に格納した例であり，$n=4$次元のデータが$m=150$個ある．

In [ ]:
from sklearn.datasets import load_iris

iris_data = load_iris()
X = iris_data["data"]

m, n = X.shape
print(f"m={m}, n={n}")
print("first 10 samples\n", X[:10], sep='')


この4つの特徴を持つデータを散布図行列でプロットしたのが以下の図である．

In [ ]:
import pandas as pd

df = pd.DataFrame(
    X,
    columns=iris_data.feature_names,
)

pd.plotting.scatter_matrix(
    df,
    c=iris_data.target,
    figsize=(10, 10),
    s=70,
    edgecolor="k",
    linewidth=0.3,
    alpha=1,
)
plt.show()


この4つの特徴は互いに関連がありそうであるが，どの特徴がどれだけ重要であるのかの判断には，
4次元データですら直感は働かない．
このような多次元データを統計的に解析する方法は多変量解析（multivariate analysis）と呼ばれており，
多数の手法が存在する．

最も広く用いられる手法の1つ主成分分析（principal component analysis, PCA）である．
これはデータの相関関係を利用して，
多変量データの特徴を最もよく表す主成分（principal component）と呼ばれる
多次元空間中の単位方向ベクトルを求める．



ある$n$次元データを表すベクトル
$$
\boldsymbol{x} = (x_{1}, x_{2}, \ldots, x_{n})^T \in R^n
$$
が与えられたとして，その各要素がどれだけ重要であるかを表す重みを
$$
\boldsymbol{w} = (w_{1}, w_{2}, \ldots, w_{n})^T \in R^n
$$
とする．そして，この重みで重みづけた新しい変数を
\begin{align*}
z &= w_1 x_{1} + w_2 x_{2} + \cdots + w_n x_{n} \\
&= \boldsymbol{w}^T \boldsymbol{x}
\end{align*}
とする．
ただし重みは正規化してあるとする．つまり
\begin{align*}
\| \boldsymbol{w} \|_2^2 = 1
\end{align*}
とする．

このときどのような$z$であれば，つまりどのような重み$\boldsymbol{w}$
（もっと言えば$R^n$の単位方向ベクトル）を用いれば，
データ$\boldsymbol{x}$を解析するために有用であるかの定義は，手法によって異なる．
主成分分析では，
データ$\boldsymbol{x}_1, \boldsymbol{x}_2, \ldots, \boldsymbol{x}_m$に対する変換後の変数
$z_1, z_2, \ldots, z_m$が最も散らばる（分散が大きくなる）方向を
$\boldsymbol{w}$として採用し，それを第1主成分（1st principal component）や第1主成分方向と呼ぶ．
なお$z_i$は$\boldsymbol{x}_i$の$\boldsymbol{w}$への射影であり，
主成分得点や主成分スコアなどと呼ばれる．


In [ ]:
x = X[0]
print("x", x)

rng = np.random.default_rng()
w = rng.random(size=(X.shape[1]))
w /= np.linalg.norm(w)
print("w", w)

z = w @ x
print("z", z)



これは$z_i$の分散$\sigma^2$を最大化する最適化問題を解くことになる．
$$
\sigma^2 = \frac{1}{m} \sum_{i=1}^m (z_i - \bar{z})^2
$$
ここで$\bar{z}$は$z_i$の平均である．
\begin{align*}
\bar{z}
&= \frac{1}{m} \sum_{i=1}^m z_i \\
&= \frac{1}{m} \sum_{i=1}^m \boldsymbol{w}^T \boldsymbol{x}_i \\
&= \boldsymbol{w}^T \bar{\boldsymbol{x}}
\end{align*}
ここで$\bar{\boldsymbol{x}}$はデータの平均ベクトルである．
\begin{align*}
\bar{\boldsymbol{x}} &= \frac{1}{m} \sum_{i=1}^m \boldsymbol{x}_i
\end{align*}
実際には$\bar{\boldsymbol{x}} = \boldsymbol{0}, \bar{z} = 0$となるように，
データから平均を除去しておくことが多い．


In [ ]:
x_bar = X.mean(axis=0)
print("x_bar", x_bar)

z_bar = w @ x_bar
print("z_bar", z_bar)



これを用いると，
\begin{align*}
\sigma^2
&= \frac{1}{m-1} \sum_{i=1}^m (z_i - \bar{z})^2 \\
&= \frac{1}{m-1} \sum_{i=1}^m (\boldsymbol{w}^T \boldsymbol{x}_i - \boldsymbol{w}^T \bar{\boldsymbol{x}})^2 \\
&= \frac{1}{m-1} \sum_{i=1}^m \boldsymbol{w}^T (\boldsymbol{x}_i - \bar{\boldsymbol{x}})(\boldsymbol{x}_i - \bar{\boldsymbol{x}})^T \boldsymbol{w}  \\
&= \boldsymbol{w}^T S \boldsymbol{w}
\end{align*}
ここで$S$はデータの共分散行列である．
\begin{align*}
S
&= \frac{1}{m-1} \sum_{i=1}^m (\boldsymbol{x}_i - \bar{\boldsymbol{x}})(\boldsymbol{x}_i - \bar{\boldsymbol{x}})^T \\
&= \frac{1}{m-1} (X'_{\mathrm{row}})^T X'_{\mathrm{row}}
\end{align*}
なお$X'_{\mathrm{row}}$は平均行ベクトルを除去した行$\boldsymbol{x}_i - \bar{\boldsymbol{x}}$で構成したデータ行列である．


In [ ]:
S = np.cov(X, rowvar=False)
print("S\n", S, sep='')


### 1.5.1. <a id='toc1_5_1_'></a>[第1主成分](#toc0_)

したがって，最適化問題は以下のようになる．
$$
\max_{\boldsymbol{w}} \boldsymbol{w}^T S \boldsymbol{w} \quad \text{s.t.} \quad \| \boldsymbol{w} \|_2^2 = 1
$$
ラグランジュ乗数を$\lambda$とすると，ラグランジュ関数は
$$
L = \boldsymbol{w}^T S \boldsymbol{w} + \lambda (\boldsymbol{w}^T \boldsymbol{w} - 1)
$$
これを$\boldsymbol{w}$で微分して0として解くと
\begin{align*}
S \boldsymbol{w} = \lambda \boldsymbol{w}
\end{align*}
となり，つまり$\boldsymbol{w}$は$S$の固有ベクトルであることが分かる．すると
\begin{align*}
\boldsymbol{w}^T S \boldsymbol{w} = \lambda
\end{align*}
であるので，$\boldsymbol{w}^T S \boldsymbol{w}$を最大化するためには$\lambda$が最大固有値$\lambda_1$でなければならず，
したがって$\boldsymbol{w}$は$S$の第1固有ベクトルであり，これが求めるべき第1主成分である．


以下では，ランダムなベクトルで$\boldsymbol{w}^T S \boldsymbol{w}$を計算しても，第1固有ベクトルで計算したほうが値が大きいことを確認している．

In [ ]:
print("w: random")
for _ in range(10):
    w = rng.random(size=(X.shape[1]))
    w /= np.linalg.norm(w)
    print("w^T S w: ", w @ S @ w)

print("w: 1st eigen vector")
s, U = np.linalg.eigh(S)
print("w^T S w: ", U[:, -1] @ S @ U[:, -1])


### 1.5.2. <a id='toc1_5_2_'></a>[第$i$主成分](#toc0_)

$d$次元データ$\boldsymbol{x}_i$を第1主成分方向へ射影して1次元データ$z_i$が得られたが，
これだけではあまりに情報が少ない．
そこで，第1主成分$\boldsymbol{w}_1$に直交する方向の中から射影後のデータの分散が最も大きくなる方向を第2主成分$\boldsymbol{w}_2$とし，
これを求めることを考える．
同様に第1主成分$\boldsymbol{w}_1$から第$i-1$主成分$\boldsymbol{w}_{i-1}$に直交する方向の中から射影後のデータの分散が最も大きくなる方向を第$i$主成分$\boldsymbol{w}_i$とする．

証明もできるが，上記の2次形式
\begin{align*}
\boldsymbol{w}^T S \boldsymbol{w} = \lambda
\end{align*}
を考えれば，
第2主成分$\boldsymbol{w}_2$は2番目に大きい固有値に対応する固有ベクトルであり，
第$i$主成分$\boldsymbol{w}_i$は$i$番目に大きい固有値に対応する固有ベクトルであることが分かる．
これを$i=1, 2, \ldots, n$について行い，それらを並べた行列を
\begin{align*}
W = (\boldsymbol{w}_1, \boldsymbol{w}_2, \ldots, \boldsymbol{w}_n) \in R^{n \times n}
\end{align*}
とおき，主成分スコアを並べたベクトルを$\boldsymbol{z} = (z_1, z_2, \ldots, z_n)^T$と書くと，
\begin{align*}
\boldsymbol{z}_i = W^T (\boldsymbol{x}_i - \bar{\boldsymbol{x}})
\end{align*}
もしくは（平均を除去した）データ行列$X'_{\mathrm{row}}$を用いて書けば
\begin{align*}
Z = X'_{\mathrm{row}} W
\end{align*}
となる．ここで$Z$は$\boldsymbol{z}_i$を行に持つ行列である．

$i \neq j$について$\boldsymbol{w}_i$と$\boldsymbol{w}_j$は直交するように選ぶことにしているため，
$W$は直交行列であり，$R^n$の基底変換を表している．
主成分分析などの統計的解析の文脈では主軸変換とも呼ばれる．


以下では，第1固有ベクトルに直交するランダムなベクトルで$\boldsymbol{w}^T S \boldsymbol{w}$を計算しても，第2固有ベクトルで計算したほうが値が大きいことを確認している．

In [ ]:
print("w: random but orthogonal to 1st eigen vector")
for _ in range(10):
    w = rng.random(size=(X.shape[1]))
    w -= (U[:, -1] @ w) * U[:, -1]
    w /= np.linalg.norm(w)
    print("w^T S w: ", w @ S @ w, w @ U[:, -1])

print("w: 2nd eigen vector")
print("w^T S w: ", U[:, -2] @ S @ U[:, -2], U[:, -2] @ U[:, -1])


### 1.5.3. <a id='toc1_5_3_'></a>[次元削減](#toc0_)

$n$番目までの主成分スコアをすべて利用する必要はない．
主成分分析を用いた統計的解析では，
寄与の大きい（第1主成分から始まる）最初のほうの主成分を利用するだけで十分である事が多く，
とくに第1主成分と第2主成分を用いて2つの主成分スコア$z_1, z_2$を2次元プロットし，
データの分布を可視化して解釈することが多い．

またパターン認識や機械学習において$n$次元データを扱う場合，$n$が大きいことが多く，
例えば画像認識では小さい画像でも$n=256\times256=65,536$画素を含むため，
そのままでは計算量の観点からも（$O(n^2)$や$O(n^3)$の行列計算やメモリが必要），
性能の観点からも（高次元データは汎化性能が落ちることが多い）得策ではない．
そのため，$n$よりも十分小さい$p \ll n$を用いて$p$次元にデータを圧縮することがよく行われる．
これを次元削減や特徴変換と呼び，
主成分分析を用いて次元削減する場合には
$p$個の主成分スコア$z_1, \ldots, z_p$を用いることになる．
このような次元削減後のデータをパターン認識や機械学習では特徴量（feature）と呼ぶ．


次元削減をしない場合には$W$は直交行列であるため，データに$W^T$をかけて主成分スコアに変換し，それに$W$をかけると，また元のデータに戻る．
\begin{align*}
\boldsymbol{z}_i &= W^T (\boldsymbol{x}_i - \bar{\boldsymbol{x}})  \in R^m\\
W \boldsymbol{z}_i&= W W^T (\boldsymbol{x}_i - \bar{\boldsymbol{x}})
= \boldsymbol{x}_i - \bar{\boldsymbol{x}}  \in R^m
\end{align*}

次元削減をする場合は$W \in R^{m \times p}$は列直交であり，
$W W^T$は$m \times m$行列であるが単位行列ではない．そのため
\begin{align*}
\boldsymbol{z}_i &= W^T (\boldsymbol{x}_i - \bar{\boldsymbol{x}}) \in R^p
\end{align*}
に対して$W^T \in R^{p \times m}$を適用しても元のデータには戻らない．
\begin{align*}
W \boldsymbol{z}_i&= W W^T (\boldsymbol{x}_i - \bar{\boldsymbol{x}})
\neq \boldsymbol{x}_i - \bar{\boldsymbol{x}}
\end{align*}
しかしこれは，$W$の$p$個の列（主成分）が張る部分空間への射影による近似になっており，
次元$p$を固定した場合の最良近似であることが示せる．

### 1.5.4. <a id='toc1_5_4_'></a>[累積寄与率](#toc0_)

次元削減をどの程度行えばよいかは，データや応用に依存する．
一般的には累積寄与率（cumurative contribution rate）を用いて判断することが多い．

第$i$主成分に対応する固有値を$\lambda_i$とすると，
第1主成分は最も分散が大きい方向であるため，
$\lambda_1$は$\lambda_2, \lambda_3, \ldots$よりも（一般的に非常に）大きい．
もし$\lambda_i = 0$であれば
$\boldsymbol{w}_i^T S \boldsymbol{w}_i = 0$であり，
つまりその第$i$主成分への射影後の分散が0であることを意味する．
0でなくとも非常に小さければ，その主成分へ射影してもほとんどデータの分布を表していないことになる．
したがって，固有値がある程度大きい第$p$主成分までを利用すればよいことになる．

全固有値の和に対する固有値の比を寄与率と呼び，
$$
\gamma_i = \frac{\lambda_i}{\sum_{k=1}^n \lambda_k}
$$
で表す．これを第1主成分から第$p$主成分までの和を取った（累積した）ものが累積寄与率

\begin{align*}
\sum_{i=1}^p \gamma_i
=
\frac{\sum_{i=1}^p \lambda_i}{\sum_{k=1}^n \lambda_k}
\end{align*}

である．$p=n$の場合には累積寄与率は1である．

一般的には累積寄与率が90%や95%になる次元$p$までの主成分を利用することが多い．



### 1.5.5. <a id='toc1_5_5_'></a>[実装（$m > n$の場合）](#toc0_)

データ行列`X`の共分散行列を計算し，主成分を求める．

改めて計算し直そう．
まず`X`の列方向に平均を取り平均ベクトルを計算し，ブロードキャストで各データから平均ベクトルを引く．

In [ ]:
x_bar = X.mean(axis=0)
X_decentered = X - x_bar


次に共分散行列を構成する．
なお不偏分散を用いるため$\frac{1}{m-1}$で割っている．（当然`np.cov()`でよいが復習である）

In [ ]:
S = (X_decentered.T @ X_decentered) / (m - 1)

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print("S\n", S, sep='')


共分散行列`S`は実対称行列であるので，`eigh()`を用いて固有値と固有ベクトルを求める．
なお数式と一致させると分かりやすいため，
最大固有値が最初の要素`[0]`に来るように，スライシング`[::-1]`で逆順に入れ替えている．

In [ ]:
d, W = np.linalg.eigh(S)
d = d[::-1]
W = W[:, ::-1]
print("eigenvalues", d)


寄与率と累積寄与率を求める．以下では寄与率ベクトルから累積寄与率ベクトルを計算するために累積和を計算する`np.cumsum`を利用している．

- https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html

In [ ]:
with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True):
    g = d / d.sum()
    print("contribution rate           ", g)
    cum_g = np.cumsum(g)
    print("cumurative contribution rate", cum_g)


寄与率と累積寄与率をプロットすると以下のようになる．
このデータ（iris）では，第1主成分が92%を占めており，1つの軸でほとんどのデータを表せる（「説明できる」と表現する）．

In [ ]:
fig = plt.figure(figsize=(10,3))
ax = fig.subplots()

ax.plot(g, '.-', label="contribution rate")
ax.plot(cum_g, '.-', label="cumularive contribution rate")

ax.set_xlabel("i")
ax.set_ylabel("%")
ax.set_ylim(0)

ax.legend()
plt.show()


主成分スコアの行列`Z`を計算し，第1主成分と第2主成分のスコアをプロットしてみる．

In [ ]:
Z = X_decentered @ W[:, :2]
Z.shape

In [ ]:
fig = plt.figure()
ax = fig.subplots()

ax.scatter(
    Z[:, 0],
    Z[:, 1],
    c=iris_data.target,
    edgecolor="k",
    linewidth=0.5,
    s=40,
)
ax.axis("equal")
ax.set_xlabel("1st principal component direction")
ax.set_ylabel("2nd principal component direction")

plt.show()


### 1.5.6. <a id='toc1_5_6_'></a>[scikit-learnによる実装](#toc0_)

scikit-learnには主成分分析が実装されており，
`skearn.decomposition.PCA`
で利用できる．

- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

まず`PCA()`で`pca`オブジェクトを作成し，
次にデータ行列`X`を`pca.fit()`に与える．
この時点で主成分分析が行われ，内部的に主成分が保持される．
主成分スコアを計算するには`pca.transform()`にデータ行列`X`を与えると，
返り値に主成分スコア行列$Z$が得られる．

主成分を計算するデータ行列と主成分スコアを計算するデータ行列が同一であれば，
`pca.fit_transform()`で一度に行える．
この場合も，主成分を別のデータに適用するなら`pca.transform()`が利用できる．


In [ ]:
from sklearn.decomposition import PCA

fig = plt.figure()
ax = fig.subplots()

pca = PCA()

pca.fit(X)
Z = pca.transform(X)
# Z = pca.fit_transform(X)

ax.scatter(
    Z[:, 0],
    Z[:, 1],
    c=iris_data.target,
    edgecolor="k",
    linewidth=0.5,
    s=40,
)
ax.axis("equal")
ax.set_xlabel("1st principal component direction")
ax.set_ylabel("2nd principal component direction")

plt.show()


主成分分析オブジェクト`pca`には主成分`W`，固有値（分散）`d`，データ平均ベクトル`x_bar`などが保持されており，
それぞれ以下のように取得できる．なお主成分方向の向きは任意であるため，主成分ベクトルは符号を除いて同一である．

In [ ]:
pca.components_.T, W


In [ ]:
pca.explained_variance_, d


In [ ]:
pca.mean_, x_bar


In [ ]:
pca.explained_variance_ratio_, g


### 1.5.7. <a id='toc1_5_7_'></a>[実装（$m < n$の場合）：顔画像データセットの主成分分析](#toc0_)

ここではsklearnのデータセットとして準備されているOlivetti facesデータセットを用いる．

- https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html

このデータセットには400枚の顔画像があり，それぞれが4096次元のベクトルである（64x64画像を表す）．

In [ ]:
face_dataset = fetch_olivetti_faces(shuffle=False)
X = face_dataset.data

m, n = X.shape
print(f"m={m}, n={n}")


0枚目の画像は以下の通り．64x64にreshpaeしてから`imshow()`で表示する．

In [ ]:
imshow(X[0].reshape(64, 64), vmin=0, vmax=1, cmap="gray")
plt.axis('off')
plt.title("0th image")
plt.show()


400枚の画像は以下の通り．

In [ ]:
plt.figure(figsize=(20, 20))
for i, p in enumerate(X):
    plt.subplot(20, 20, i + 1)
    plt.imshow(X[i].reshape(64, 64), vmin=0, vmax=1, cmap="gray")
    plt.axis('off')


平均ベクトルは画像としてreshapeして表示すると，平均顔画像となる．


In [ ]:
x_bar = X.mean(axis=0)


In [ ]:
imshow(x_bar.reshape(64, 64), cmap="gray")
plt.axis('off')
plt.title("mean image")
plt.show()


ではこのデータで共分散行列を作成し，主成分分析のために固有値を計算することを考える．
先程のアヤメデータのような$m=140, n=4$の場合とは異なり，
ここでは$m=400, n=4096$であるため，共分散行列は$4096 \times 4096$となる．

In [ ]:
X_decentered = X - x_bar
# S = (X_decentered.T @ X_decentered) / (m - 1)
# eigh(S)


`S`を保持するためのメモリ使用量は$O(n^2)$であり，
float64では8bytes * 4092 * 4096 = 約130MBである．
計算中にメモリ領域はさらに必要となる．
この程度ならまだ計算できるかもしれないが，
一般的なサイズの画像では共分散行列をメモリに乗せることも難しく，
また$O(n^3)$の計算量も非常に大きくなる．

そのため$m \ll n$の場合には，以下のような手順を考える．

\begin{align*}
S \boldsymbol{w}_i &= \lambda_i \boldsymbol{w}_i \\
\frac{1}{m-1} X^T X \boldsymbol{w}_i &= \lambda_i \boldsymbol{w}_i \\
\frac{1}{m-1} X X^T X \boldsymbol{w}_i &= \lambda_i X \boldsymbol{w}_i \\
\left( \frac{1}{m-1} X X^T \right) (X \boldsymbol{w}_i) &= \lambda_i (X \boldsymbol{w}_i) \\
\end{align*}

つまり
$X \boldsymbol{w}_i$は
$\frac{1}{m-1} X X^T$の
固有値$\lambda_i$に対応する固有ベクトルであることが分かる．
$X X^T$は$m \times m$であるため，必要メモリ量は$O(m^2)$，計算量は$O(m^3)$となるため，
$\frac{1}{m-1} X X^T$の
固有値$\lambda_i$に対応する固有ベクトルを$\boldsymbol{v}_i \in R^m$を求めることができる．
すると，今度は逆に以下のような導出が得られる．

\begin{align*}
\frac{1}{m-1} X X^T \boldsymbol{v}_i &= \lambda_i \boldsymbol{v}_i \\
\frac{1}{m-1} X^T X X^T \boldsymbol{v}_i &= \lambda_i X^T \boldsymbol{v}_i \\
\left( \frac{1}{m-1} X^T X \right) (X^T \boldsymbol{v}_i) &= \lambda_i (X^T \boldsymbol{v}_i) \\
\end{align*}

つまり
$X^T \boldsymbol{v}_i$は
$\frac{1}{m-1} X^T X$の
固有値$\lambda_i$に対応する固有ベクトル，すなわち主成分$\boldsymbol{w}_i \in R^n$であることが分かる．
どちらの固有ベクトルも
$\| \boldsymbol{v}_i \| = \| \boldsymbol{w}_i \| = 1$と正規化されているとすると，
次式が得られる．
\begin{align*}
\boldsymbol{w}_i = \frac{1}{\sqrt{(m-1)\lambda_i}} X^T \boldsymbol{v}_i
\end{align*}

ではこの方法で，$X X^T$の固有ベクトル`V`を計算し，
$X^T X$の固有ベクトル`W`に変換する．

In [ ]:
S = (X_decentered @ X_decentered.T) / (m - 1)
d, V = eigh(S)

d = d[::-1]
V = V[:, ::-1]

W = X_decentered.T @ (V / np.sqrt(d * (m - 1)))  # using broadcast


これはPCAの主成分に一致する．

In [ ]:
pca = PCA()
pca.fit(X)


ただし計算精度の問題で，`np.allclose()`ではTrueにならないが，ノルムを計算するとほぼ一致していることが分かる．

In [ ]:
print("1st pc == w_0", np.allclose(pca.components_[0], W[:, 0]))
print("|| 1st pc == w_0 ||", norm(pca.components_[0] - W[:, 0]))

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print(pca.components_[0])
    print(W[:, 0])


ここで$X \in R^{m \times n}$についての特異値分解
$$
X = U \Sigma V^T
$$
を考えると，
\begin{align*}
X^T X &= V \Sigma U^T U \Sigma V^T = V \Sigma^2 V^T \in R^{n \times n} \\
X X^T &= U \Sigma V^T V \Sigma U^T = U \Sigma^2 U^T \in R^{m \times m}
\end{align*}
であり，つまり
$X^T X$の固有ベクトル$W$は
$X$の右特異ベクトル$V$であることが分かり，
それを求めるためには
$X$の左特異ベクトル$U$を求めれば良いことが分かる．

`sklearn.decomposition.PCA`などの実際の主成分分析の実装の内部では，
特異値分解を用いて主成分を計算している．

- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
    - https://github.com/scikit-learn/scikit-learn/blob/093e0cf14aff026cca6097e8c42f83b735d26358/sklearn/decomposition/_pca.py#L607

In [ ]:
U, s, Vt = np.linalg.svd(X_decentered, full_matrices=False)


In [ ]:
print("w_0 == vt_0", np.allclose(W[:, 0], Vt[0]))

with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
    print(W[:, 0])
    print(Vt[0])


#### 1.5.7.1. <a id='toc1_5_7_1_'></a>[固有顔](#toc0_)

主成分$\boldsymbol{w}_i$はデータ$\boldsymbol{x}_i$と同じ$n$次元ベクトルであり，
データが画像の場合には，画像として表示することができる．
顔画像の場合にはこれを固有顔（eigenface）と呼び，
顔画像データの認識や解析において用いられていた．

以下は平均顔と，第5主成分までの固有顔を表示した例である．
これをみると，第1主成分は画像全体の大きな明暗の変動を表しており，
第2主成分は左右の照明変化を，
第3主成分は上下方向の照明変化を表していると解釈できる．


In [ ]:
p = 5
plt.figure(figsize=(p * 2, 5))

plt.subplot(1, p + 1, 1)
plt.imshow(pca.mean_.reshape(64, 64),cmap="gray")
plt.axis('off')
plt.title(f"mean face", fontsize=8)

for i in range(p):
    plt.subplot(1, p + 1, i + 2)
    plt.imshow(pca.components_[i].reshape(64, 64), cmap="gray")
    plt.axis('off')
    plt.title(f"{i + 1}-th eigenface", fontsize=8)

plt.show()


この顔画像データでは，累積寄与率が90%になるには第50主成分程度が必要なことが分かる．

In [ ]:
fig = plt.figure(figsize=(10, 3))
ax = fig.subplots()

ax.plot(pca.explained_variance_ratio_, '-', label="contribution rate")
ax.plot(np.cumsum(pca.explained_variance_ratio_), '-', label="cumularive contribution rate")

ax.set_xlabel("i")
ax.set_ylabel("%")
ax.set_ylim(0, 1)

ax.legend()
plt.show()


## 1.6. <a id='toc1_6_'></a>[課題](#toc0_)

### 1.6.1. <a id='toc1_6_1_'></a>[課題05-1：QR法による固有値分解](#toc0_)

QR法の手順は，
ハウスホルダー変換により三重対角行列へ変換し，
ギブンス変換によるQR分解を反復するものだった．
これにより全ての固有値と固有ベクトル（の近似値）が求められる．

しかしQR法の反復の収束条件として，主対角成分が変化しなくなるだけでは精度が悪いため（まだ非対角成分が0に収束していない），
優対角成分が変化しなくなるまでとしていた．しかしその分，QR法の反復回数が増えていた．

そこで，QR法の反復の収束条件には主対角成分が変化しなくなるまでとし，
その時点での近似固有値（主対角成分）と近似固有ベクトルを初期値とするレイリー商反復を用いて，
各固有値と固有ベクトルを更新することを考える．

1. この方法で`A`の固有値分解（対角化）を行う処理を，1つの関数関数`my_eigh()`として作成せよ．
    - 返り値は`A`の固有値からなるベクトルと，固有ベクトルからなる直交行列とする（`np.linalg.eigh()`と同様）．
2. 得られた固有値と固有ベクトルが，`np.linalg.eigh()`の結果と一致していることを確かめよ（一致していない場合は原因を考えよ）．


### 1.6.2. <a id='toc1_6_2_'></a>[課題05-2：ハウスホルダー変換によるQR分解](#toc0_)

#### 1.6.2.1. <a id='toc1_6_2_1_'></a>[「一般の行列を三重対角行列に変換する」ためのハウスホルダー法の復習](#toc0_)

「一般の行列を三重対角行列に変換する」ハウスホルダー法では，
以下のように，鏡映変換$Q_1$を左から$A$にかけて，$A$の第1列の第3要素以下を0にすることを考えた．

\begin{align*}
Q_1
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
a_{31} & a_{32} & \cdots & a_{3n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{pmatrix}
=
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
* & * & \cdots & * \\
0 & * & \cdots & * \\
\vdots & \vdots & \ddots & \vdots \\
0 & * & \cdots & *
\end{pmatrix}
\end{align*}

三重対角行列に変換するためにはこのような$Q_{\mathrm{hh}} = Q_1 Q_2 \cdots Q_{n-2}$を左右からかけて

$$
A_{\mathrm{tri}} = Q_{\mathrm{hh}}^T A Q_{\mathrm{hh}}
$$

と$A$を三重対角行列へ変換した．


#### 1.6.2.2. <a id='toc1_6_2_2_'></a>[「一般の行列をQR分解する」ためのハウスホルダー変換](#toc0_)

これと同じ要領で「一般の行列をQR分解する」ためにハウスホルダー変換を行うには，
以下のように，ハウスホルダー行列$H_1$を左から$A$にかけて，$A$の第1列の第2要素以下を0にすることを考える．

\begin{align*}
H_1
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
a_{31} & a_{32} & \cdots & a_{3n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{pmatrix}
=
\begin{pmatrix}
* & * & \cdots & * \\
0 & * & \cdots & * \\
0 & * & \cdots & * \\
\vdots & \vdots & \ddots & \vdots \\
0 & * & \cdots & *
\end{pmatrix}
\end{align*}

三重対角行列に変換するためにはこのような$H_1, H_2, \cdots, H_{n-1}$を左からかけて

\begin{align*}
H_{n-1} \cdots H_2 H_1 A &= R \\
A &= (H_{n-1} \cdots H_2 H_1)^{-1} R \\
&= (H_1 H_2 \cdots H_{n-1}) R \\
&= QR
\end{align*}

と$A$をQR分解する．


#### 1.6.2.3. <a id='toc1_6_2_3_'></a>[課題](#toc0_)

1. 上記の手順で`A`のQR分解を実装し，1つの関数関数`my_qr_decomp()`として作成せよ．返り値は直交行列`Q`と上三角行列`R`とする．
2. 得られたQR分解が`np.linalg.qr`（または`scipy.linalg.qr`）の結果と一致していることを確かめよ（一致していない場合は原因を考えよ）．

### 1.6.3. <a id='toc1_6_3_'></a>[課題05-3：SVDによる主成分分析](#toc0_)

行列$A \in R^{m \times n}$の特異値分解$A=U \Sigma V^T$を計算する以下の単純な方法を関数`my_svd()`として実装し，
以下の2つの場合の数値例を計算せよ（プロット・画像なども比較せよ）．

$m > n$の場合：
1. $A^T A$に対して対角化を計算し，$V^T (A^T A) V = \Sigma^2$を得る．
1. $A V$をQR分解して$AV = UR$を得る．
1. SVD結果$A = U R V^T$を出力する．返り値は特異値の順番も含めて`np.linalg.svd`と同様の形式とする．

$m < n$の場合：
1. $A A^T$に対して対角化を計算し，$U^T (A A^T) U = \Sigma^2$を得る．
1. $A^T U$をQR分解して$A^T U = VR$を得る．
1. SVD結果$A = U R V^T$を出力する．返り値は特異値の順番も含めて`np.linalg.svd`と同様の形式とする．

条件：

- QR分解には`my_qr_decomp()`を使用すること．
  - ただし`np.linalg.qr()`と結果が一致していることを確認すること．
- 対角化には`my_eigh()`を使用すること．
  - ただし`np.linalg.eigh()`と結果が一致していることを確認すること．
- `my_svd()`の結果が`np.linalg.svd()`の結果と一致することを確認すること．
- 数値例のデータには，$m > n$の場合はアヤメのデータセットを，$m < n$の場合は顔画像のデータセットを利用すること．
    - 主成分分析としての出力は`sklearn.decomposition.PCA`の結果と一致することを確かめよ（一致していない場合は原因を考えよ）．
